In [9]:

# SEAG qualification period is 22 Oct 2024 to 5 Sept 2025

#1. Segregate into Male and Femal 
#2. For each gender perform the following: 
#a. Sort data by mapped eent, then perf scalar (higher the better)
#b. Identify tiers based on performance - Tier 1 is meets bronze medal mark for SEAG, Tier 2 is 2% and Tier 3 is 3.5%
#c. Check - if athlete met bronze or 2%/3.5% then delta_benchmark is zero or +, delta2% is + and delta 3.5% is +
#d. Top ranked athletes for each event are chosen. Max number of athletes for each event is 3, except for 100m/400m which is 6
#    This includes athletes on spex scholarship and potential
#e. The max for each tier is 2. Lower ranked athletes move down one tier.
#3. If athlete qualifies for more than one event the higher tier event is given
#4. Jump and throws junior program to be solved separately

%load_ext autoreload
%autoreload 2

In [45]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters
import datetime
import icecream as ic
import dateutil.parser as parser 
import datacompy
import pytz

from itertools import permutations



from google.cloud import storage



In [21]:
'''
# PRODUCTION ENVIRONMENT
# Extract timed event records

import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT NAME, RESULT, TEAM, AGE, RANK AS COMPETITION_RANK, DIVISION, EVENT, DISTANCE, EVENT_CLASS, UNIQUE_ID, DOB, NATIONALITY, WIND, CATEGORY_EVENT, GENDER, COMPETITION, DATE, YEAR, REGION, TIMESTAMP
FROM `saa-analytics.results.athlete_results_prod` 
WHERE RESULT!='NM' AND RESULT!='-' AND RESULT!='DNS' AND RESULT!='DNF' AND RESULT!='DNQ' AND RESULT!='DQ' AND RESULT IS NOT NULL

"""

competitors = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)


'''

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [81]:
# PRODUCTION ENVIRONMENT
# Extract timed event records

import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT NAME, RESULT, TEAM, AGE, RANK AS COMPETITION_RANK, DIVISION, EVENT, DISTANCE, EVENT_CLASS, UNIQUE_ID, DOB, NATIONALITY, WIND, CATEGORY_EVENT, GENDER, COMPETITION, DATE, YEAR, REGION, TIMESTAMP
FROM `saa-analytics.results.PRODUCTION` 
WHERE RESULT!='NM' AND RESULT!='-' AND RESULT!='DNS' AND RESULT!='DNF' AND RESULT!='DNQ' AND RESULT!='DQ' AND RESULT IS NOT NULL

"""

competitors = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)



Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [82]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION,TIMESTAMP
0,Shanti Veronica Pereira,11.71,nan,nan,7.0,nan,100m,nan,nan,nan,20-Sep-96,SGP,-1.8,Sprint,Female,AtleticaGenève,2025-06-21 00:00:00+00:00,2025,International,2025-06-29 22:00:26.182128
1,Shanti Veronica Pereira,23.16w,nan,nan,7.0,nan,200m,nan,nan,nan,20-Sep-96,SGP,2.4,Sprint,Female,AtleticaGenève,2025-06-21 00:00:00+00:00,2025,International,2025-06-29 22:00:26.182128
2,Jun Yu Low,4.77,nan,nan,5.0,nan,Pole Vault,nan,nan,nan,21-Apr-01,SGP,nan,Jump,Male,Jan Dietvorst Memorial,2025-06-21 00:00:00+00:00,2025,International,2025-06-29 22:00:26.182128
3,Tung Hon Andrew Pak,2.05,nan,nan,nan,nan,High Jump,nan,nan,nan,10-Apr-02,SGP,nan,Jump,Male,National Championships,2025-06-25 00:00:00+00:00,2025,International,2025-06-29 22:00:26.182128
4,Tam Jong Hng,1.95,nan,nan,nan,nan,High Jump,nan,nan,nan,nan,SGP,nan,Jump,Male,National Championships,2025-06-25 00:00:00+00:00,2025,International,2025-06-29 22:00:26.182128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31280,Charlene Lee,8.90,NYGH,None,15,C,Triple Jump,None,None,None,None,None,None,Jump,Female,National School Games,2024-04-01 00:00:00+00:00,2024,Local,None
31281,"Ng Jing Wen, Ariel",10.33,RGS,None,2,C,Triple Jump,None,None,None,None,None,None,Jump,Female,National School Games,2024-04-01 00:00:00+00:00,2024,Local,None
31282,LEE GABRIEL JIN YI,14.5,SINGAPORE,21,3,None,Triple jump,None,None,None,2003,SGP,NWI,Jump,Male,Victor Saneev Memorial (Georgia),2024-10-26 00:00:00+00:00,2024,International,None
31283,"SYED AHMED RIADH, SHARIFAH FALISHA",11.13,SINGAPORE SPORTS SCHOOL,18,None,None,Triple jump,None,None,None,2006,SGP,NWI,Jump,Female,FTKLAA State Meet,2024-03-03 00:00:00+00:00,2024,International,None


In [83]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


competitors.to_csv('athletes_new_format_prod.csv', sep=',', encoding='utf-8-sig', index=False)

In [84]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION,TIMESTAMP
0,Shanti Veronica Pereira,11.71,nan,nan,7.0,nan,100m,nan,nan,nan,20-Sep-96,SGP,-1.8,Sprint,Female,AtleticaGenève,2025-06-21 00:00:00+00:00,2025,International,2025-06-29 22:00:26.182128
1,Shanti Veronica Pereira,23.16w,nan,nan,7.0,nan,200m,nan,nan,nan,20-Sep-96,SGP,2.4,Sprint,Female,AtleticaGenève,2025-06-21 00:00:00+00:00,2025,International,2025-06-29 22:00:26.182128
2,Jun Yu Low,4.77,nan,nan,5.0,nan,Pole Vault,nan,nan,nan,21-Apr-01,SGP,nan,Jump,Male,Jan Dietvorst Memorial,2025-06-21 00:00:00+00:00,2025,International,2025-06-29 22:00:26.182128
3,Tung Hon Andrew Pak,2.05,nan,nan,nan,nan,High Jump,nan,nan,nan,10-Apr-02,SGP,nan,Jump,Male,National Championships,2025-06-25 00:00:00+00:00,2025,International,2025-06-29 22:00:26.182128
4,Tam Jong Hng,1.95,nan,nan,nan,nan,High Jump,nan,nan,nan,nan,SGP,nan,Jump,Male,National Championships,2025-06-25 00:00:00+00:00,2025,International,2025-06-29 22:00:26.182128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31280,Charlene Lee,8.90,NYGH,None,15,C,Triple Jump,None,None,None,None,None,None,Jump,Female,National School Games,2024-04-01 00:00:00+00:00,2024,Local,None
31281,"Ng Jing Wen, Ariel",10.33,RGS,None,2,C,Triple Jump,None,None,None,None,None,None,Jump,Female,National School Games,2024-04-01 00:00:00+00:00,2024,Local,None
31282,LEE GABRIEL JIN YI,14.5,SINGAPORE,21,3,None,Triple jump,None,None,None,2003,SGP,NWI,Jump,Male,Victor Saneev Memorial (Georgia),2024-10-26 00:00:00+00:00,2024,International,None
31283,"SYED AHMED RIADH, SHARIFAH FALISHA",11.13,SINGAPORE SPORTS SCHOOL,18,None,None,Triple jump,None,None,None,2006,SGP,NWI,Jump,Female,FTKLAA State Meet,2024-03-03 00:00:00+00:00,2024,International,None


In [85]:
competitors[competitors['COMPETITION']=='Box Hill Burn']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION,TIMESTAMP
43,Jayden Tan,02:31.1,None,None,5,None,1000m,None,None,None,24-Jun-06,SGP,None,Mid,Male,Box Hill Burn,2025-03-06 00:00:00+00:00,2025,International,None
44,Romaine Soh,02:56.0,None,None,7,None,1000m,None,None,None,26-Nov-94,SGP,None,Mid,Female,Box Hill Burn,2025-03-06 00:00:00+00:00,2025,International,None
7462,Shaun Goh,14:57.4,None,None,3,None,5000m,None,None,None,12-Jan-97,SGP,None,Long,Male,Box Hill Burn,2025-03-06 00:00:00+00:00,2025,International,None
7463,Vanessa Lee Ying Zhuang,17:06.7,None,None,11,None,5000m,None,None,None,23-Feb-98,SGP,None,Long,Female,Box Hill Burn,2025-03-06 00:00:00+00:00,2025,International,None
7464,Nicole Low,17:21.3,None,None,13,None,5000m,None,None,None,19-Jun-98,SGP,None,Long,Female,Box Hill Burn,2025-03-06 00:00:00+00:00,2025,International,None


In [86]:
'''
# Create mm/dd/yy column for date of event

import datetime

#s=247.779

#datetime.datetime.fromtimestamp(s).strftime('%M:%S.%f')

for i in range(len(competitors)):
        
    rowIndex = competitors.index[i]

    date = competitors.loc[rowIndex,'DATE']
    year = competitors.loc[rowIndex,'YEAR']    
    
    if 'to' in date or ' - ' in date:
        
        if re.search('to|\s\-\s\d\s|\s\-\d\d', date):  # e.g. 03-04
              
            pos = re.search('to|\s\-\s\d', date)
            # Splice string to day and month

            split_pos_start=pos.start()+3
            


            final_date = date[split_pos_start:] # left string post splicing

            print(i, pos, date, final_date)
            final_year = year[2:]

            event_date = final_date + '/' + final_year

            print('old code', date, final_date, event_date)

            competitors.loc[rowIndex, 'event_date'] = event_date

        elif re.search('(\-\s\d\w)|(\-\s\d\d\w)', date):  # e.g. 18 - 19 January
                        
            pos = re.search('\-', date)  # from '-' onwards only
            # Splice string to day and month

            split_pos_start=pos.start()+2
            

            final_date = date[split_pos_start:] # left string post splicing

            
            final_year = year[2:]

            event_date = final_date + ' ' + final_year


            competitors.loc[rowIndex, 'event_date'] = event_date
            
        
    elif re.search('\w\-\w', date):
        
        if competitors.loc[rowIndex, 'COMPETITION'] == "National School Games":
            
            if competitors.loc[rowIndex, 'YEAR'] == '2024':
        
                event_date = '04'+'/'+date[1:3] + '/' + year[2:]  # reverse order from dd/mm to mm/dd. 04 because event was in April 24 only
            
       #         print('NSG 2024', event_date)
        
                competitors.loc[rowIndex, 'event_date'] = event_date
            
            elif competitors.loc[rowIndex, 'YEAR'] == '2025':
                
                event_date = date + '-' +year[2:]
                
        #        print('NSG2025', event_date)
                
                competitors.loc[rowIndex, 'event_date'] = event_date
                
        elif re.search('\d\-\d',  date):        #10-13 April
            
            print('HERE', i, date)

            rpos = re.search('\-', date)
            string = date[rpos.end():]
            
            print('extracted date', string)
            
            event_date = string + ' ' + year
            
            print('event date', event_date)
            
            competitors.loc[rowIndex, 'event_date'] = event_date

            
        
        else:
            
            event_date = date + '-' + year[2:]
            
            competitors.loc[rowIndex, 'event_date'] = event_date

    else:
            
            event_date = date + '-' + year[2:]
            
            competitors.loc[rowIndex, 'event_date'] = event_date

        

        
competitors['event_date'] = competitors['event_date'].astype(str)
competitors['event_date'] = competitors['event_date'].str.replace('\xa0', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('\r', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('\n', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.strip()
       
'''    

'\n# Create mm/dd/yy column for date of event\n\nimport datetime\n\n#s=247.779\n\n#datetime.datetime.fromtimestamp(s).strftime(\'%M:%S.%f\')\n\nfor i in range(len(competitors)):\n        \n    rowIndex = competitors.index[i]\n\n    date = competitors.loc[rowIndex,\'DATE\']\n    year = competitors.loc[rowIndex,\'YEAR\']    \n    \n    if \'to\' in date or \' - \' in date:\n        \n        if re.search(\'to|\\s\\-\\s\\d\\s|\\s\\-\\d\\d\', date):  # e.g. 03-04\n              \n            pos = re.search(\'to|\\s\\-\\s\\d\', date)\n            # Splice string to day and month\n\n            split_pos_start=pos.start()+3\n            \n\n\n            final_date = date[split_pos_start:] # left string post splicing\n\n            print(i, pos, date, final_date)\n            final_year = year[2:]\n\n            event_date = final_date + \'/\' + final_year\n\n            print(\'old code\', date, final_date, event_date)\n\n            competitors.loc[rowIndex, \'event_date\'] = event_date\n

In [87]:
#assert competitors['event_date'].all()!='nan' # make sure the conversion to datetime format is valid
#assert str(competitors['event_date'].all())!='NaT' # make sure the conversion to datetime format is valid


In [88]:
'''

# Calculate number of days from today to event date

competitors['event_date_dt'] = pd.to_datetime(competitors['event_date'], format='mixed', dayfirst=False)

competitors['delta_time']= datetime.datetime.now() - competitors['event_date_dt']

competitors['delta_time_conv'] = pd.to_numeric(competitors['delta_time'].dt.days, downcast='integer')

competitors['event_month'] = competitors['event_date_dt'].dt.month
'''

"\n\n# Calculate number of days from today to event date\n\ncompetitors['event_date_dt'] = pd.to_datetime(competitors['event_date'], format='mixed', dayfirst=False)\n\ncompetitors['delta_time']= datetime.datetime.now() - competitors['event_date_dt']\n\ncompetitors['delta_time_conv'] = pd.to_numeric(competitors['delta_time'].dt.days, downcast='integer')\n\ncompetitors['event_month'] = competitors['event_date_dt'].dt.month\n"

In [100]:
# Date field to contain timezone

competitors['DATE'] = pd.to_datetime(competitors['DATE'], format='mixed', dayfirst=False, utc=True)


In [101]:
competitors['DATE']

0       2025-06-21 00:00:00+00:00
1       2025-06-21 00:00:00+00:00
2       2025-06-21 00:00:00+00:00
3       2025-06-25 00:00:00+00:00
4       2025-06-25 00:00:00+00:00
                   ...           
31280   2024-04-01 00:00:00+00:00
31281   2024-04-01 00:00:00+00:00
31282   2024-10-26 00:00:00+00:00
31283   2024-03-03 00:00:00+00:00
31284   2024-07-06 00:00:00+00:00
Name: DATE, Length: 31285, dtype: datetime64[us, UTC]

In [102]:
# datetime to contain UTC (timezone)

competitors['NOW'] = datetime.datetime.now()

timezone = pytz.timezone('UTC')

competitors['NOW'] = datetime.datetime.now().replace(tzinfo=timezone)

In [103]:
competitors['NOW']

0       2025-07-02 21:12:17.012255+00:00
1       2025-07-02 21:12:17.012255+00:00
2       2025-07-02 21:12:17.012255+00:00
3       2025-07-02 21:12:17.012255+00:00
4       2025-07-02 21:12:17.012255+00:00
                      ...               
31280   2025-07-02 21:12:17.012255+00:00
31281   2025-07-02 21:12:17.012255+00:00
31282   2025-07-02 21:12:17.012255+00:00
31283   2025-07-02 21:12:17.012255+00:00
31284   2025-07-02 21:12:17.012255+00:00
Name: NOW, Length: 31285, dtype: datetime64[us, UTC]

In [104]:
# Calculate number of days from today to event date

competitors['DATE'] = pd.to_datetime(competitors['DATE'], format='mixed', dayfirst=False, utc=False)

competitors['delta_time'] = competitors['NOW'] - competitors['DATE']


#competitors['delta_time'] = datetime.datetime.now() - competitors['DATE']


competitors['delta_time_conv'] = pd.to_numeric(competitors['delta_time'].dt.days, downcast='integer')

competitors['event_month'] = competitors['DATE'].dt.month


In [105]:
competitors['delta_time']

0        11 days 21:12:17.012255
1        11 days 21:12:17.012255
2        11 days 21:12:17.012255
3         7 days 21:12:17.012255
4         7 days 21:12:17.012255
                  ...           
31280   457 days 21:12:17.012255
31281   457 days 21:12:17.012255
31282   249 days 21:12:17.012255
31283   486 days 21:12:17.012255
31284   361 days 21:12:17.012255
Name: delta_time, Length: 31285, dtype: timedelta64[us]

In [106]:
# Make sure date conversion is is valid for all rows

assert not competitors['delta_time'].isna().any()

In [107]:
# These results have not had their event dates converted properly

competitors[competitors['DATE'].isna()]

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,GENDER,COMPETITION,DATE,YEAR,REGION,TIMESTAMP,NOW,delta_time,delta_time_conv,event_month


In [108]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,GENDER,COMPETITION,DATE,YEAR,REGION,TIMESTAMP,NOW,delta_time,delta_time_conv,event_month
0,Shanti Veronica Pereira,11.71,nan,nan,7.0,nan,100m,nan,nan,nan,...,Female,AtleticaGenève,2025-06-21 00:00:00+00:00,2025,International,2025-06-29 22:00:26.182128,2025-07-02 21:12:17.012255+00:00,11 days 21:12:17.012255,11,6
1,Shanti Veronica Pereira,23.16w,nan,nan,7.0,nan,200m,nan,nan,nan,...,Female,AtleticaGenève,2025-06-21 00:00:00+00:00,2025,International,2025-06-29 22:00:26.182128,2025-07-02 21:12:17.012255+00:00,11 days 21:12:17.012255,11,6
2,Jun Yu Low,4.77,nan,nan,5.0,nan,Pole Vault,nan,nan,nan,...,Male,Jan Dietvorst Memorial,2025-06-21 00:00:00+00:00,2025,International,2025-06-29 22:00:26.182128,2025-07-02 21:12:17.012255+00:00,11 days 21:12:17.012255,11,6
3,Tung Hon Andrew Pak,2.05,nan,nan,nan,nan,High Jump,nan,nan,nan,...,Male,National Championships,2025-06-25 00:00:00+00:00,2025,International,2025-06-29 22:00:26.182128,2025-07-02 21:12:17.012255+00:00,7 days 21:12:17.012255,7,6
4,Tam Jong Hng,1.95,nan,nan,nan,nan,High Jump,nan,nan,nan,...,Male,National Championships,2025-06-25 00:00:00+00:00,2025,International,2025-06-29 22:00:26.182128,2025-07-02 21:12:17.012255+00:00,7 days 21:12:17.012255,7,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31280,Charlene Lee,8.90,NYGH,None,15,C,Triple Jump,None,None,None,...,Female,National School Games,2024-04-01 00:00:00+00:00,2024,Local,None,2025-07-02 21:12:17.012255+00:00,457 days 21:12:17.012255,457,4
31281,"Ng Jing Wen, Ariel",10.33,RGS,None,2,C,Triple Jump,None,None,None,...,Female,National School Games,2024-04-01 00:00:00+00:00,2024,Local,None,2025-07-02 21:12:17.012255+00:00,457 days 21:12:17.012255,457,4
31282,LEE GABRIEL JIN YI,14.5,SINGAPORE,21,3,None,Triple jump,None,None,None,...,Male,Victor Saneev Memorial (Georgia),2024-10-26 00:00:00+00:00,2024,International,None,2025-07-02 21:12:17.012255+00:00,249 days 21:12:17.012255,249,10
31283,"SYED AHMED RIADH, SHARIFAH FALISHA",11.13,SINGAPORE SPORTS SCHOOL,18,None,None,Triple jump,None,None,None,...,Female,FTKLAA State Meet,2024-03-03 00:00:00+00:00,2024,International,None,2025-07-02 21:12:17.012255+00:00,486 days 21:12:17.012255,486,3


In [111]:
# Choose date range for SEAG qualification window from Oct 22 to current


competitors['DATE']=competitors['DATE'].dt.tz_localize(None)  # switch off timezone for compatibility with np.datetime64

start = datetime.datetime(2024, 6, 23)
#start = datetime.datetime(2025, 5, 1)


end = datetime.datetime(2025, 6, 10)

start_date = np.datetime64(start)
end_date = np.datetime64(end)


mask = (competitors['DATE'] >= start_date) & (competitors['DATE'] <= end_date)
athletes_selected = competitors.loc[mask]



In [112]:
athletes_selected.to_csv('athletes_downloaded_AY.csv', encoding='utf-8')

In [3016]:
# Select all of 2024/25 for OCTC

#athletes_selected = competitors[(competitors['YEAR']=='2024')|(competitors['YEAR']=='2025')]

#athletes_selected = competitors[(competitors['YEAR']=='2025')]

In [113]:
athletes_selected

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,GENDER,COMPETITION,DATE,YEAR,REGION,TIMESTAMP,NOW,delta_time,delta_time_conv,event_month
23,SOH RUI YONG GUILLAUME,04:59.3,INDIVIDUAL,33,12,None,1 mile,None,None,None,...,Male,Vitaly West Mininster Mile 5K (UK),2024-09-22,2024,International,None,2025-07-02 21:12:17.012255+00:00,283 days 21:12:17.012255,283,9
25,Shaun Goh,32:21.0,None,None,16,None,"10,000m",None,None,None,...,Male,26th Asian Athletics Championships,2025-05-31,2025,International,2025-06-01 16:49:06.835826,2025-07-02 21:12:17.012255+00:00,32 days 21:12:17.012255,32,5
26,VANESSA LEE YING ZHUANG,40:21:00,SINGAPORE,26,10,None,10000m,None,None,None,...,Female,Asian Cross Country (Hong Kong),2024-10-20,2024,International,None,2025-07-02 21:12:17.012255+00:00,255 days 21:12:17.012255,255,10
27,DARREN CHONG SHUN HAO,37:54:00,SINGAPORE,31,25,None,10000m,10000m,None,None,...,Male,Asian Cross Country (Hong Kong),2024-10-20,2024,International,None,2025-07-02 21:12:17.012255+00:00,255 days 21:12:17.012255,255,10
28,SOH RUI YONG GUILLAUME,31:11.4,INDIVIDUAL,33,133,None,10000m,None,None,None,...,Male,"Tokai University Long Distance Challenge 10,00...",2024-11-25,2024,International,None,2025-07-02 21:12:17.012255+00:00,219 days 21:12:17.012255,219,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31180,"GOH, YEN YOUNG, AMELIA",10.14m,DUNMAN HIGH SCHOOL,16,3,U18,Triple Jump,0,None,None,...,Female,16th SEA Youth Invitational Trial,2024-09-08,2024,Local,None,2025-07-02 21:12:17.012255+00:00,297 days 21:12:17.012255,297,9
31181,"Chew, Keng Yue Joshua",12.06m,Eunoia Junior College,17,4,U18,Triple Jump,0,None,None,...,Male,16th SEA Youth Invitational Trial,2024-09-08,2024,Local,None,2025-07-02 21:12:17.012255+00:00,297 days 21:12:17.012255,297,9
31182,"LAU, REINA",9.80m,Singapore Sports School,17,4,U18,Triple Jump,0,None,None,...,Female,16th SEA Youth Invitational Trial,2024-09-08,2024,Local,None,2025-07-02 21:12:17.012255+00:00,297 days 21:12:17.012255,297,9
31282,LEE GABRIEL JIN YI,14.5,SINGAPORE,21,3,None,Triple jump,None,None,None,...,Male,Victor Saneev Memorial (Georgia),2024-10-26,2024,International,None,2025-07-02 21:12:17.012255+00:00,249 days 21:12:17.012255,249,10


In [3018]:
athletes_selected[athletes_selected['NAME']=='Hia, Caleb']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,TIMESTAMP,event_date,event_date_dt,delta_time,delta_time_conv,event_month


In [3019]:
# Choose 2024/25 only

athletes = athletes_selected

In [3020]:
athletes

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,TIMESTAMP,event_date,event_date_dt,delta_time,delta_time_conv,event_month
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 17:01:10.066819,11.0,6.0
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 17:01:10.066819,11.0,6.0
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 17:01:10.066819,11.0,6.0
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 17:01:10.066819,11.0,6.0
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 17:01:10.066819,11.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31154,St. Andrew's Junior College,03:43.06,SAJC,None,5.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,52 days 17:01:10.066819,52.0,4.0
31155,Catholic Junior College,03:43.67,CJC,None,6.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,52 days 17:01:10.066819,52.0,4.0
31156,Yishun Innova Junior College,03:45.03,YIJC,None,7.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,52 days 17:01:10.066819,52.0,4.0
31157,National Junior College,03:47.40,NJC,None,8.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,52 days 17:01:10.066819,52.0,4.0


In [3021]:
athletes[athletes['NAME']=='Jun Jie Calvin Quek']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,TIMESTAMP,event_date,event_date_dt,delta_time,delta_time_conv,event_month
18,Jun Jie Calvin Quek,50.94,None,None,8,None,400m Hurdles,None,None,None,...,26th Asian Athletics Championships,31-May,2025,International,36:03.8,31-May-25,2025-05-31,19 days 17:01:10.066819,19.0,5.0
31,Jun Jie Calvin Quek,50.58,None,None,2,None,400m Hurdles,None,None,None,...,26th Asian Athletics Championships,27 - 31 May,2025,International,2025-06-01 16:49:06.835826,31 May 25,2025-05-31,19 days 17:01:10.066819,19.0,5.0
4333,Jun Jie Calvin Quek,51.64,None,None,6.0,None,400m Hurdles,None,None,None,...,12th Kinami Michitaka Memorial Athletics Meet,11-May,2025,International,2025-05-16 15:18:11.207945,11-May-25,2025-05-11,39 days 17:01:10.066819,39.0,5.0
5424,Jun Jie Calvin Quek,51.79,None,None,2,None,400m Hurdles,None,None,None,...,XXXIII Qosanov Memorial,22 - 23 June,2024,International,None,23 June 24,2024-06-23,361 days 17:01:10.066819,361.0,6.0
7485,Jun Jie Calvin Quek,10.86,None,None,2,None,100m,None,None,None,...,Potch Invitational Meet,16-Apr,2025,International,None,16-Apr-25,2025-04-16,64 days 17:01:10.066819,64.0,4.0
19361,Jun Jie Calvin Quek,50.77,None,None,1,None,400m Hurdles,None,None,None,...,Potch Invitational Meet,16-Apr,2025,International,None,16-Apr-25,2025-04-16,64 days 17:01:10.066819,64.0,4.0
22617,Jun Jie Calvin Quek,52.62,None,None,9,None,400m Hurdles,None,None,None,...,Sydney Track Classic,15-Mar,2025,International,None,15-Mar-25,2025-03-15,96 days 17:01:10.066819,96.0,3.0
23631,Jun Jie Calvin Quek,56.95,None,None,3,None,400m Hurdles,None,None,None,...,XXXIII Qosanov Memorial,22 - 23 June,2024,International,None,23 June 24,2024-06-23,361 days 17:01:10.066819,361.0,6.0


In [3022]:
# Process dates to extract age

# Map NSG divisions into age

mask = (athletes['DIVISION'].str.contains(r'A', na=False))
athletes.loc[mask, 'AGE'] = '18.5'

mask = (athletes['DIVISION'].str.contains(r'B', na=False))
athletes.loc[mask, 'AGE'] = '16'

mask = (athletes['DIVISION'].str.contains(r'C', na=False))
athletes.loc[mask, 'AGE'] = '13.5'

mask = (athletes['DIVISION'].str.contains(r'O', na=False))
athletes.loc[mask, 'AGE'] = '12'


In [3023]:
def length(string):

    B = ''
    year = ''

    try:

        length = len(string)

        if length == 2:

            string = '19' + string

        elif length == 1:

            string = ''

        else:

            pass

        if string is not None or len(string) != 1:

            B = parser.parse(string, dayfirst=True)
                        
    except:

        pass

    return B


athletes['DOB_dt'] = athletes['DOB'].apply(length)




/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/1704523743.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes['DOB_dt'] = athletes['DOB'].apply(length)


In [3024]:
'''
athletes['DOB_new'] = pd.to_datetime(athletes['DOB_new'], errors='coerce')

athletes['year_extract'] = athletes['DOB_new'].dt.strftime('%Y')

athletes['year_extract'] = pd.to_numeric(athletes['year_extract'])

athletes['age_extract'] = 2025 - athletes['year_extract']
'''

"\nathletes['DOB_new'] = pd.to_datetime(athletes['DOB_new'], errors='coerce')\n\nathletes['year_extract'] = athletes['DOB_new'].dt.strftime('%Y')\n\nathletes['year_extract'] = pd.to_numeric(athletes['year_extract'])\n\nathletes['age_extract'] = 2025 - athletes['year_extract']\n"

In [3025]:
'''
def age(number):  # correct negative age numbers
    
    if number<0:
        
        number+=100
        
    return number


athletes['age_extract']=athletes['age_extract'].apply(age)
'''

"\ndef age(number):  # correct negative age numbers\n    \n    if number<0:\n        \n        number+=100\n        \n    return number\n\n\nathletes['age_extract']=athletes['age_extract'].apply(age)\n"

In [3026]:
'''
# If NSG event then choose AGE otherwise choose age_extract

condition1 = athletes['COMPETITION']=='National School Games'
#condition2=((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))
#condition3=df['SEED_CONV']<df['RESULT_CONV']
#condition4=~((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))


athletes['age_extract'] = athletes['AGE'].where((condition1), athletes['age_extract'].values)

athletes[['age_extract']] = athletes[['age_extract']].apply(pd.to_numeric)
'''

"\n# If NSG event then choose AGE otherwise choose age_extract\n\ncondition1 = athletes['COMPETITION']=='National School Games'\n#condition2=((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))\n#condition3=df['SEED_CONV']<df['RESULT_CONV']\n#condition4=~((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))\n\n\nathletes['age_extract'] = athletes['AGE'].where((condition1), athletes['age_extract'].values)\n\nathletes[['age_extract']] = athletes[['age_extract']].apply(pd.to_numeric)\n"

In [3027]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


athletes.to_csv('age_post_map_AY.csv', sep=',', encoding='utf-8-sig', index=False)


In [3028]:
# Create temporary mapped event column
# Junior Category Only

athletes['MAPPED_EVENT']=''

for col in athletes.columns:
    athletes[col] = athletes[col].astype(str)
    athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
    athletes[col] = athletes[col].str.strip()


# Correct javelin category

mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
athletes.loc[mask, 'CATEGORY_EVENT'] = 'Throw'


# Running

mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'100', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'100', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'100 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'^100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'

mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'200', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'200', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'^200m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'200\sMeter', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'

mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'400', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'^400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'^400\sMeter$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'400', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'


mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'800', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'800 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'^800m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'1000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1000m'


mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'1500', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes['EVENT'].str.contains(r'^1500m$', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'3000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
#mask = athletes['EVENT'].str.contains(r'3000m', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'5000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes['EVENT'].str.contains(r'^5000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'10000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = athletes['EVENT'].str.contains(r'^10000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = athletes['EVENT'].str.contains(r'^10\,000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'Mile', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1 Mile'

#mask = athletes['EVENT'].str.contains(r'10\,000m', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m'



# Hurdles

#mask = athletes['EVENT'].str.contains(r'100\sMeter\sHurdles\s\(0\.838m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
#mask = athletes['EVENT'].str.contains(r'100m\sHurdles\s\(0\.838m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'


mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(0\.914m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'
##mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(1\.067m\)', na=True)
##athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'


mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains('0.762', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
#mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['DIVISION'].str.contains('None', na=False) & athletes['GENDER'].str.contains(r'Female', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
#athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'100', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
#mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'



#mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
#mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.838|0.84', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
#mask = ((athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False)) 
#         & ((athletes['EVENT_CLASS'].str.contains('None', na=False))|(athletes['EVENT_CLASS']==np.nan)|(athletes['EVENT_CLASS']=='')) 
#         & athletes['REGION'].str.contains(r'International', na=False) & (athletes['DIVISION'].str.contains(r'None', na=False)))  # this is the correct syntax
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'

#mask = (athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'


# Using np.where instead

#athletes['MAPPED_EVENT'] = np.where(((athletes['EVENT']=='110m hurdles|110m Hurdles') & ((athletes['EVENT_CLASS']=='')|athletes['EVENT_CLASS']=='None') & (athletes['REGION']=='International')), '110m Hurdles', ' ')   
                                
mask = (athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914|91.4', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'


mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['DIVISION'].str.contains(r'U18', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'

#mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(' ', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'







#mask = (athletes['EVENT'].str.contains(r'^400m\sHurdles$', na=False) & athletes['EVENT_CLASS'].str.contains(r'', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
#mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0.840m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = ' '
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762|76.2cm', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


#mask = athletes['EVENT'].str.contains(r'400\sMeter\sHurdles\s\(0\.914m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
#mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0\.914m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.84|84|0.838', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['DIVISION'].str.contains(r'U18', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'

mask = (athletes['EVENT'].str.contains(r'400m Hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.84|84|0.838', na=False)  & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


#mask = athletes['EVENT'].str.contains(r'^400\sMeter\sHurdles\s\(0\.762m\)$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
##mask = athletes['EVENT'].str.contains(r'^400m\sHurdles\s\(0\.762m\)$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False)& athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762m', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains('0.762|0.84|0.838', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
#mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'



# Throws


#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(600g\)', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s600g', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s600g\)', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(800g\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'

mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['EVENT_CLASS'].str.contains(r'500g', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['EVENT_CLASS'].str.contains(r'700g', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True) & (athletes['GENDER']=='Female') & (athletes['EVENT_CLASS']=='3kg'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'


#mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s4kg\sOpen', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Men\sShot\sPut\s4kg\sOPEN', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'


#mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s\(4kg\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Shot\sPut\s\(7\.26kg\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Shot\sPut\s7\.26kg\sOpen', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['GENDER']=='Male') & (athletes['EVENT_CLASS'].str.contains(r'5', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['GENDER']=='Female') & (athletes['EVENT_CLASS'].str.contains(r'3', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'

#mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False)))# there are some additional characters after Put
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'

#mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['REGION'].str.contains(r'International', na=False)) & athletes['EVENT_CLASS'].str.contains(r'None', na=False))# there are some additional characters after Put
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'



#mask = (athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(4kg\)', na=True) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
#mask = athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(7\.26kg\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'5kg', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'3.00kg', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
#mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False)))# there are some additional characters after Put
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'



#mask = ((athletes['EVENT'].str.contains(r'Discus\sThrow\s\(1kg\)', na=False)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

#mask = ((athletes['EVENT'].str.contains(r'Discus\s\(1\.00kg\)', na=False))  & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'


#mask = athletes['EVENT'].str.contains(r'Discus\sThrow\s\(2kg\)', na=False)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
#mask = ((athletes['EVENT'].str.contains(r'Discus\sThrow\s\(1kg\)', na=False)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'1.5kg|1.50kg', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'1kg|1.00kg', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

#mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['REGION'].str.contains(r'International', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['DIVISION'].str.contains(r'U18', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
#mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['DIVISION'].str.contains(r'None', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

#mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'



# Jumps

mask = athletes['EVENT'].str.contains(r'High Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'

mask = athletes['EVENT'].str.contains(r'^Long\sJump$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Open', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Trial', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'


mask = athletes['EVENT'].str.contains(r'Triple Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'Pole Vault', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'
mask = athletes['EVENT'].str.contains(r'High jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'
mask = athletes['EVENT'].str.contains(r'Long jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Triple jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'^Pole\svault$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'

# Steeplechase

#mask = athletes['EVENT'].str.contains(r'2000m S/C', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = athletes['EVENT'].str.contains(r'2000m steeplechase', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = athletes['EVENT'].str.contains(r'2000 Meter Steeplechase', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = (athletes['EVENT'].str.contains(r'2000m Steeplechase|2000m S\/C', na=True) & athletes['REGION'].str.contains(r'International', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'Steeplechase|steeplechase|S\/C', na=False) & athletes['DISTANCE'].str.contains(r'2000', na=False)  & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False)  & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'Steeplechase|steeplechase|S\/C', na=False) & athletes['DISTANCE'].str.contains(r'2000', na=False)  & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False)  & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'


mask = (athletes['EVENT'].str.contains(r'2000m Steeplechase|2000m steeplechase|2000m S\/C', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'2000m Steeplechase|2000m steeplechase|2000m S\/C', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'


#mask = (athletes['EVENT'].str.contains(r'Steeplechase', na=False) & athletes['DISTANCE'].str.contains(r'2000', na=False)  & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'


# Marathon

mask = athletes['EVENT'].str.contains(r'^Marathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\sMarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\smarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'


# Walk

#mask = athletes['EVENT'].str.contains(r'1500m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'
#mask = athletes['EVENT'].str.contains(r'1500 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'1500', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'


#mask = athletes['EVENT'].str.contains(r'3000m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'
#mask = athletes['EVENT'].str.contains(r'3000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'


#mask = athletes['EVENT'].str.contains(r'5000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Race Walk'
#mask = athletes['EVENT'].str.contains(r'5000m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Race Walk'
mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'5000', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Racewalk'

mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'10000', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m Racewalk'


#mask = athletes['EVENT'].str.contains(r'10000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m Race Walk'

# Relay

mask = athletes['EVENT'].str.contains(r'4x80m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 80m'
mask = athletes['EVENT'].str.contains(r'^4\sx\s100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = athletes['EVENT'].str.contains(r'4x100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = athletes['EVENT'].str.contains(r'4 X 100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = (athletes['EVENT'].str.contains(r'Relay', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'

mask = athletes['EVENT'].str.contains(r'4x400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'4 X 400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'4x100 Meter Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = (athletes['EVENT'].str.contains(r'Relay', na=False) & athletes['DISTANCE'].str.contains(r'1600', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'^4\sx\s400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'

# Decathlon/Heptathlon

mask = athletes['EVENT'].str.contains(r'^Heptathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'^Decathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'
mask = athletes['EVENT'].str.contains(r'Heptathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'Decathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'



/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3513542181.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes['MAPPED_EVENT']=''
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3513542181.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].astype(str)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3513542181.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3513542181.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3513542181.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3513542181.py:12: SettingWithCopyWarning: 
A value is tryin

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3513542181.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].astype(str)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3513542181.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3513542181.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [3029]:
for col in athletes.columns:
    athletes[col] = athletes[col].astype(str)
    athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
    athletes[col] = athletes[col].str.strip()

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2203170124.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].astype(str)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2203170124.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2203170124.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2203170124.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2203170124.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2203170124.py:6: SettingWithCopyWarning: 
A 

In [3030]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


athletes.to_csv('athletes_post_map_AY.csv', sep=',', encoding='utf-8-sig', index=False)


In [3031]:
# Load Asian Youth Benchmarks

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Benchmarks/')

all_benchmark = pd.read_csv("All_Benchmarks.csv")




In [3032]:
all_benchmark

,BENCHMARK,EVENT,GENDER,RESULT
0,26th Asian Athletics,100m,Male,10.33
1,26th Asian Athletics,100m,Female,11.66
2,26th Asian Athletics,200m,Male,21.01
3,26th Asian Athletics,200m,Female,23.66
4,26th Asian Athletics,400m,Male,46.29
...,...,...,...,...
159,2025 Asian Youth,5000m Race Walk,Female,28:58.25
160,2025 Asian Youth,Decathlon,Male,5249.00
161,2025 Asian Youth,Heptathlon,Female,3965.00
162,2025 Asian Youth,Medley Relay,Male,01:59.16


In [3033]:
benchmark=all_benchmark[all_benchmark['BENCHMARK']=='2025 Asian Youth']

In [3034]:
benchmark

,BENCHMARK,EVENT,GENDER,RESULT
124,2025 Asian Youth,100m,Male,10.85
125,2025 Asian Youth,100m,Female,12.25
126,2025 Asian Youth,200m,Male,22.25
127,2025 Asian Youth,200m,Female,25.00
128,2025 Asian Youth,400m,Male,49.75
129,2025 Asian Youth,400m,Female,59.12
130,2025 Asian Youth,800m,Male,02:00.12
131,2025 Asian Youth,800m,Female,02:23.21
132,2025 Asian Youth,1500m,Male,04:08.48
133,2025 Asian Youth,1500m,Female,04:53.05


In [3035]:
benchmark=benchmark.reset_index(drop=True)

In [3036]:
# Converts any time format into seconds

def convert_time(i, string, metric):

    global output
    
    l=['discus', 'throw', 'jump', 'vault', 'shot']
        
    string=string.lower()
    
   # print('metric', metric)
    
    try:
        
        if 'w' in metric:  # skip marks with illegal wind speeds
            
        #    print('W', metric)
            
            output=''
            
        else:
            
    
            if any(s in string for s in l)==True:
            
                if 'm' in metric:
            
                    metric=metric.replace('m', '')
                    output=float(str(metric))
            
                elif 'GR' in metric:
            
                    metric=metric.replace('GR', '')
                    output=float(str(metric))
                
                
                else:
    
                    output=float(str(metric))
        
            elif string=='':   # no event description at all!
                
                output='' # return nothing
            
                
        
            else:
        
                searchstring = ":"
                searchstring2 = "."
                substring=str(metric)
                count = substring.count(searchstring)
                count2 = substring.count(searchstring2)
            
                if count==0:
                
                    output=float(substring)
            
            
                elif '10,000m' in string and count==2:  # fix erroneous timing format from XX:XX:XX to XX:XX.XX
                
                
                    idx = 5 # 6th character position
                    replacement = "."
                    metric = metric[:idx] + replacement + metric[idx+1:]                
                
                    m,s = metric.split(':')            

                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())

                elif '5000m' in string and count==2:  # fix erroneous timing format from XX:XX:XX to XX:XX.XX
                
                
                    idx = 5 # 6th character position
                    replacement = "."
                    metric = metric[:idx] + replacement + metric[idx+1:]                
                
                    m,s = metric.split(':')            

                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())

                    
                    
                elif '1500m' in string and count==2:  # fix erroneous timing format from XX:XX:XX to XX:XX.XX
                    
                    if len(substring)==7:  # format is X:XX:XX and not XX:XX:XX 
                        
                        idx = 4 # 5th character position
                        replacement = "."
                        metric = '0' + metric[:idx] + replacement + metric[idx+1:]                
                
                        m,s = metric.split(':')            

                        output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
                    
                        
                    else:  # format is XX:XX:XX
                        
                        idx = 5 # 5th character position
                        replacement = "."
                        metric = metric[:idx] + replacement + metric[idx+1:]                
                
                        m,s = metric.split(':')            

                        output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())  
             
                elif (type(metric)==datetime.time or type(metric)==datetime.datetime):
                
                                                
                    time=str(metric)
                    h, m ,s = time.split(':')
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                                
                elif (count==1 and count2==1):
            
                    m,s = metric.split(':')
                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
                     
                elif (count==1 and count2==2):
                
            
                    metric = metric.replace(".", ":", 1)
            
                    h,m,s = metric.split(':')            
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
                
        
                elif (count==2 and count2==0):
                
            
                    h,m,s = metric.split(':')
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
  
            

    except:
        
        pass
                
    return output

In [3037]:
def process_benchmarks(df):
    
    for i in range(len(df)):

        rowIndex = df.index[i]

        input_string=df.iloc[rowIndex,0]
    
        metric=df.iloc[rowIndex,3]
    
        if metric==None:
        
            continue
        
        out = convert_time(i, input_string, metric)
        
        print(rowIndex, input_string, out)

    
        df.loc[rowIndex, 'Metric'] = out
    
    return df

In [3038]:
process_benchmarks(benchmark)

0 2025 Asian Youth 10.85
1 2025 Asian Youth 12.25
2 2025 Asian Youth 22.25
3 2025 Asian Youth 25.0
4 2025 Asian Youth 49.75
5 2025 Asian Youth 59.12
6 2025 Asian Youth 120.12
7 2025 Asian Youth 143.21
8 2025 Asian Youth 248.48
9 2025 Asian Youth 293.05
10 2025 Asian Youth 551.87
11 2025 Asian Youth 656.29
12 2025 Asian Youth 386.83
13 2025 Asian Youth 468.16
14 2025 Asian Youth 14.25
15 2025 Asian Youth 15.53
16 2025 Asian Youth 56.39
17 2025 Asian Youth 67.36
18 2025 Asian Youth 1.97
19 2025 Asian Youth 1.62
20 2025 Asian Youth 6.94
21 2025 Asian Youth 5.57
22 2025 Asian Youth 13.63
23 2025 Asian Youth 11.88
24 2025 Asian Youth 16.69
25 2025 Asian Youth 12.74
26 2025 Asian Youth 49.94
27 2025 Asian Youth 33.99
28 2025 Asian Youth 52.57
29 2025 Asian Youth 33.6
30 2025 Asian Youth 58.78
31 2025 Asian Youth 40.61
32 2025 Asian Youth 4.03
33 2025 Asian Youth 3.08
34 2025 Asian Youth 2875.22
35 2025 Asian Youth 1738.25
36 2025 Asian Youth 5249.0
37 2025 Asian Youth 3965.0
38 2025 Asian Yo

,BENCHMARK,EVENT,GENDER,RESULT,Metric
0,2025 Asian Youth,100m,Male,10.85,10.85
1,2025 Asian Youth,100m,Female,12.25,12.25
2,2025 Asian Youth,200m,Male,22.25,22.25
3,2025 Asian Youth,200m,Female,25.00,25.00
4,2025 Asian Youth,400m,Male,49.75,49.75
5,2025 Asian Youth,400m,Female,59.12,59.12
6,2025 Asian Youth,800m,Male,02:00.12,120.12
7,2025 Asian Youth,800m,Female,02:23.21,143.21
8,2025 Asian Youth,1500m,Male,04:08.48,248.48
9,2025 Asian Youth,1500m,Female,04:53.05,293.05


In [3039]:
'''
for i in range(len(benchmarks)):
        
    rowIndex = benchmarks.index[i]

    input_string=benchmarks.iloc[rowIndex,0]
    
    metric=benchmarks.iloc[rowIndex,3]
    
    if metric==None:
        continue
        
    out = convert_time(i, input_string, metric)
    
    print(rowIndex, input_string, out)
     
    benchmarks.loc[rowIndex, 'Metric'] = out
'''

"\nfor i in range(len(benchmarks)):\n        \n    rowIndex = benchmarks.index[i]\n\n    input_string=benchmarks.iloc[rowIndex,0]\n    \n    metric=benchmarks.iloc[rowIndex,3]\n    \n    if metric==None:\n        continue\n        \n    out = convert_time(i, input_string, metric)\n    \n    print(rowIndex, input_string, out)\n     \n    benchmarks.loc[rowIndex, 'Metric'] = out\n"

In [3040]:
benchmark

,BENCHMARK,EVENT,GENDER,RESULT,Metric
0,2025 Asian Youth,100m,Male,10.85,10.85
1,2025 Asian Youth,100m,Female,12.25,12.25
2,2025 Asian Youth,200m,Male,22.25,22.25
3,2025 Asian Youth,200m,Female,25.00,25.00
4,2025 Asian Youth,400m,Male,49.75,49.75
5,2025 Asian Youth,400m,Female,59.12,59.12
6,2025 Asian Youth,800m,Male,02:00.12,120.12
7,2025 Asian Youth,800m,Female,02:23.21,143.21
8,2025 Asian Youth,1500m,Male,04:08.48,248.48
9,2025 Asian Youth,1500m,Female,04:53.05,293.05


In [3041]:
mask = benchmark['EVENT'].str.contains(r'jump|throw|Pole|put|Jump|Throw|pole|Put|Decathlon|Heptathlon', na=True)

benchmark.loc[mask, '2%']=benchmark['Metric']*0.98
benchmark.loc[mask, '3.5%']=benchmark['Metric']*0.965
benchmark.loc[mask, '5%']=benchmark['Metric']*0.95

benchmark.loc[~mask, '2%']=benchmark['Metric']*1.02
benchmark.loc[~mask, '3.5%']=benchmark['Metric']*1.035
benchmark.loc[~mask, '5%']=benchmark['Metric']*1.05


#benchmarks.iloc[5, [1]]='10000m run'
#benchmarks.iloc[28, [1]]='10000m run'
#benchmarks.iloc[26, [1]]='1500m'


In [3042]:
benchmark['MAPPED_EVENT']=benchmark['EVENT'].str.strip()

In [3043]:
for col in benchmark.columns:
    benchmark[col] = benchmark[col].astype(str)
    benchmark[col] = benchmark[col].str.replace('\xa0', ' ', regex=True)
    benchmark[col] = benchmark[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    benchmark[col] = benchmark[col].str.replace('\r', ' ', regex=True)
    benchmark[col] = benchmark[col].str.replace('\n', ' ', regex=True)
    benchmark[col] = benchmark[col].str.strip()


In [3044]:
benchmark

,BENCHMARK,EVENT,GENDER,RESULT,Metric,2%,3.5%,5%,MAPPED_EVENT
0,2025 Asian Youth,100m,Male,10.85,10.85,11.067,11.22975,11.3925,100m
1,2025 Asian Youth,100m,Female,12.25,12.25,12.495000000000001,12.678749999999999,12.8625,100m
2,2025 Asian Youth,200m,Male,22.25,22.25,22.695,23.02875,23.3625,200m
3,2025 Asian Youth,200m,Female,25.00,25.0,25.5,25.874999999999996,26.25,200m
4,2025 Asian Youth,400m,Male,49.75,49.75,50.745,51.491249999999994,52.237500000000004,400m
5,2025 Asian Youth,400m,Female,59.12,59.12,60.3024,61.18919999999999,62.076,400m
6,2025 Asian Youth,800m,Male,02:00.12,120.12,122.5224,124.32419999999999,126.126,800m
7,2025 Asian Youth,800m,Female,02:23.21,143.21,146.07420000000002,148.22235,150.37050000000002,800m
8,2025 Asian Youth,1500m,Male,04:08.48,248.48,253.4496,257.17679999999996,260.904,1500m
9,2025 Asian Youth,1500m,Female,04:53.05,293.05,298.911,303.30674999999997,307.70250000000004,1500m


In [3045]:
athletes

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,YEAR,REGION,TIMESTAMP,event_date,event_date_dt,delta_time,delta_time_conv,event_month,DOB_dt,MAPPED_EVENT
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 17:01:10.066819,11.0,6.0,1996-06-23 00:00:00,800m
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 17:01:10.066819,11.0,6.0,,5000m
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 17:01:10.066819,11.0,6.0,2003-02-23 00:00:00,Triple Jump
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 17:01:10.066819,11.0,6.0,,Pole Vault
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 17:01:10.066819,11.0,6.0,1994-11-26 00:00:00,800m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31154,St. Andrew's Junior College,03:43.06,SAJC,18.5,5.0,A,4x400m Relay,None,None,None,...,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,52 days 17:01:10.066819,52.0,4.0,,4 x 400m
31155,Catholic Junior College,03:43.67,CJC,18.5,6.0,A,4x400m Relay,None,None,None,...,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,52 days 17:01:10.066819,52.0,4.0,,4 x 400m
31156,Yishun Innova Junior College,03:45.03,YIJC,18.5,7.0,A,4x400m Relay,None,None,None,...,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,52 days 17:01:10.066819,52.0,4.0,,4 x 400m
31157,National Junior College,03:47.40,NJC,18.5,8.0,A,4x400m Relay,None,None,None,...,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,52 days 17:01:10.066819,52.0,4.0,,4 x 400m


In [3046]:
# There is a problem with RESULTS column being changed after this statement

#df = athletes.reset_index().merge(benchmarks.reset_index(), on=['MAPPED_EVENT','GENDER'], how='left')
#df = athletes.merge(benchmarks, on=['EVENT','GENDER'], how='left')


In [3047]:
# Merge benchmarks onto athletes on MAPPED_EVENT and GENDER

df = pd.merge(
    left=athletes, 
    right=benchmark,
    how='left',
    left_on=['MAPPED_EVENT', 'GENDER'],
    right_on=['MAPPED_EVENT', 'GENDER'],
)

In [3048]:
df

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,event_month,DOB_dt,MAPPED_EVENT,BENCHMARK,EVENT_y,RESULT_y,Metric,2%,3.5%,5%
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,6.0,1996-06-23 00:00:00,800m,2025 Asian Youth,800m,02:00.12,120.12,122.5224,124.32419999999999,126.126
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,6.0,,5000m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,6.0,2003-02-23 00:00:00,Triple Jump,2025 Asian Youth,Triple Jump,13.63,13.63,13.3574,13.15295,12.948500000000001
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,6.0,,Pole Vault,2025 Asian Youth,Pole Vault,3.08,3.08,3.0184,2.9722,2.9259999999999997
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,6.0,1994-11-26 00:00:00,800m,2025 Asian Youth,800m,02:23.21,143.21,146.07420000000002,148.22235,150.37050000000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,St. Andrew's Junior College,03:43.06,SAJC,18.5,5.0,A,4x400m Relay,None,None,None,...,4.0,,4 x 400m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17329,Catholic Junior College,03:43.67,CJC,18.5,6.0,A,4x400m Relay,None,None,None,...,4.0,,4 x 400m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17330,Yishun Innova Junior College,03:45.03,YIJC,18.5,7.0,A,4x400m Relay,None,None,None,...,4.0,,4 x 400m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17331,National Junior College,03:47.40,NJC,18.5,8.0,A,4x400m Relay,None,None,None,...,4.0,,4 x 400m,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3049]:
# replace '-' with NaN

df['RESULT_x'] = df['RESULT_x'].replace(regex=r'–', value=np.NaN)
#df['SEED'] = df['SEED'].replace(regex=r'–', value=np.NaN)


In [3050]:
df

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,event_month,DOB_dt,MAPPED_EVENT,BENCHMARK,EVENT_y,RESULT_y,Metric,2%,3.5%,5%
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,6.0,1996-06-23 00:00:00,800m,2025 Asian Youth,800m,02:00.12,120.12,122.5224,124.32419999999999,126.126
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,6.0,,5000m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,6.0,2003-02-23 00:00:00,Triple Jump,2025 Asian Youth,Triple Jump,13.63,13.63,13.3574,13.15295,12.948500000000001
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,6.0,,Pole Vault,2025 Asian Youth,Pole Vault,3.08,3.08,3.0184,2.9722,2.9259999999999997
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,6.0,1994-11-26 00:00:00,800m,2025 Asian Youth,800m,02:23.21,143.21,146.07420000000002,148.22235,150.37050000000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,St. Andrew's Junior College,03:43.06,SAJC,18.5,5.0,A,4x400m Relay,None,None,None,...,4.0,,4 x 400m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17329,Catholic Junior College,03:43.67,CJC,18.5,6.0,A,4x400m Relay,None,None,None,...,4.0,,4 x 400m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17330,Yishun Innova Junior College,03:45.03,YIJC,18.5,7.0,A,4x400m Relay,None,None,None,...,4.0,,4 x 400m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17331,National Junior College,03:47.40,NJC,18.5,8.0,A,4x400m Relay,None,None,None,...,4.0,,4 x 400m,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3051]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


df.to_csv('Asian_Youth_postmap.csv', sep=',', encoding='utf-8-sig', index=False)


In [3052]:
# Convert results and seed into seconds format

df.reset_index(drop=True, inplace=True)

for col in df.columns:
    
    df[col] = df[col].astype(str)
    df[col] = df[col].str.replace('\xa0', ' ', regex=True)
    df[col] = df[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    df[col] = df[col].str.replace('\r', ' ', regex=True)
    df[col] = df[col].str.replace('\n', ' ', regex=True)
    df[col] = df[col].str.strip()

for i in range(len(df)):
    
    result_out=''
    
        
    rowIndex = df.index[i]

    event=df.loc[rowIndex,'MAPPED_EVENT']    # event description
    
    result=df.loc[rowIndex,'RESULT_x'] # result
    
    if result=='—' or result=='DQ' or result=='SCR' or result=='FS' or result=='DNQ' or result=='DNS' or result=='NH' or result=='NM' or result=='FOUL' or result=='DNF' or result=='SR' :
        continue
    
    result_out = convert_time(i, event, result)
         
    df.loc[rowIndex, 'RESULT_CONV'] = result_out



/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/201318515.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[rowIndex, 'RESULT_CONV'] = result_out


In [3053]:
#df["AGE"].fillna(0, inplace=True)
#df['AGE'] = df['AGE'].astype('float')

In [3054]:
df

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,DOB_dt,MAPPED_EVENT,BENCHMARK,EVENT_y,RESULT_y,Metric,2%,3.5%,5%,RESULT_CONV
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,1996-06-23 00:00:00,800m,2025 Asian Youth,800m,02:00.12,120.12,122.5224,124.32419999999999,126.126,114.0
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,,5000m,nan,nan,nan,nan,nan,nan,nan,1114.34
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,2003-02-23 00:00:00,Triple Jump,2025 Asian Youth,Triple Jump,13.63,13.63,13.3574,13.15295,12.948500000000001,15.15
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,,Pole Vault,2025 Asian Youth,Pole Vault,3.08,3.08,3.0184,2.9722,2.9259999999999997,3.2
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,1994-11-26 00:00:00,800m,2025 Asian Youth,800m,02:23.21,143.21,146.07420000000002,148.22235,150.37050000000002,136.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,St. Andrew's Junior College,03:43.06,SAJC,18.5,5.0,A,4x400m Relay,None,None,None,...,,4 x 400m,nan,nan,nan,nan,nan,nan,nan,223.06
17329,Catholic Junior College,03:43.67,CJC,18.5,6.0,A,4x400m Relay,None,None,None,...,,4 x 400m,nan,nan,nan,nan,nan,nan,nan,223.67
17330,Yishun Innova Junior College,03:45.03,YIJC,18.5,7.0,A,4x400m Relay,None,None,None,...,,4 x 400m,nan,nan,nan,nan,nan,nan,nan,225.03
17331,National Junior College,03:47.40,NJC,18.5,8.0,A,4x400m Relay,None,None,None,...,,4 x 400m,nan,nan,nan,nan,nan,nan,nan,227.4


In [3055]:
# Choose SEED if better than RESULT

#condition1=df['SEED_CONV']>df['RESULT_CONV']
#condition2=((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))
#condition3=df['SEED_CONV']<df['RESULT_CONV']
#condition4=~((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))


#df['RESULT_BEST']=df['SEED_CONV'].where((condition1 & condition2)|(condition3 & condition4), df['RESULT_CONV'].values)

df['RESULT_BEST'] = df['RESULT_CONV']

In [3056]:
df

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,MAPPED_EVENT,BENCHMARK,EVENT_y,RESULT_y,Metric,2%,3.5%,5%,RESULT_CONV,RESULT_BEST
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,800m,2025 Asian Youth,800m,02:00.12,120.12,122.5224,124.32419999999999,126.126,114.0,114.0
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,5000m,nan,nan,nan,nan,nan,nan,nan,1114.34,1114.34
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,Triple Jump,2025 Asian Youth,Triple Jump,13.63,13.63,13.3574,13.15295,12.948500000000001,15.15,15.15
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,Pole Vault,2025 Asian Youth,Pole Vault,3.08,3.08,3.0184,2.9722,2.9259999999999997,3.2,3.2
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,800m,2025 Asian Youth,800m,02:23.21,143.21,146.07420000000002,148.22235,150.37050000000002,136.9,136.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,St. Andrew's Junior College,03:43.06,SAJC,18.5,5.0,A,4x400m Relay,None,None,None,...,4 x 400m,nan,nan,nan,nan,nan,nan,nan,223.06,223.06
17329,Catholic Junior College,03:43.67,CJC,18.5,6.0,A,4x400m Relay,None,None,None,...,4 x 400m,nan,nan,nan,nan,nan,nan,nan,223.67,223.67
17330,Yishun Innova Junior College,03:45.03,YIJC,18.5,7.0,A,4x400m Relay,None,None,None,...,4 x 400m,nan,nan,nan,nan,nan,nan,nan,225.03,225.03
17331,National Junior College,03:47.40,NJC,18.5,8.0,A,4x400m Relay,None,None,None,...,4 x 400m,nan,nan,nan,nan,nan,nan,nan,227.4,227.4


In [3057]:
# Change to numeric

df[['2%', '3.5%', '5%', 'RESULT_BEST', 'Metric']] = df[['2%', '3.5%', '5%', 'RESULT_BEST', 'Metric']].apply(pd.to_numeric, errors='coerce')

In [3058]:
mask = df['CATEGORY_EVENT'].str.contains(r'Jump|Throw|jump|throw|Decathlon|Heptathlon|decathlon|heptathlon', na=True)

df.loc[mask, 'Delta2'] = df['RESULT_BEST']-df['2%']
df.loc[mask, 'Delta3.5'] = df['RESULT_BEST']-df['3.5%']
df.loc[mask, 'Delta5'] = df['RESULT_BEST']-df['5%']
df.loc[mask, 'Delta_Benchmark'] = df['RESULT_BEST']-df['Metric']

df.loc[~mask, 'Delta2'] =  df['2%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta3.5'] = df['3.5%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta5'] = df['5%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta_Benchmark'] = df['Metric'] - df['RESULT_BEST']

#rslt_df['Delta2']=rslt_df['2pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta35']=rslt_df['35pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta5']=rslt_df['5pc']-rslt_df['RESULT_CONV']
df=df.loc[df['COMPETITION']!='Southeast Asian Games'] # Do not include results from SEAG in dataset

In [3059]:
# Performance metric to filter out athletes

df['PERF_SCALAR']=df['Delta5']/df['Metric']*100

In [3060]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


df.to_csv('AY_postmap_benchmarked.csv', sep=',', encoding='utf-8-sig', index=False)


In [3061]:
df[df['MAPPED_EVENT']=='10,000m']

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
25,Shaun Goh,32:21.0,None,None,16,None,"10,000m",None,None,None,...,NaN,NaN,NaN,1941.0,1941.0,NaN,NaN,NaN,NaN,NaN
311,DARREN CHONG SHUN HAO,37:54:00,SINGAPORE,31,25,None,10000m,10000m,None,None,...,NaN,NaN,NaN,2274.0,2274.0,NaN,NaN,NaN,NaN,NaN
509,"Heng Chin Kiat, Richard",34:42.3,National University Singapore,12,2,Open,Run,10000,None,nan,...,NaN,NaN,NaN,2082.3,2082.3,NaN,NaN,NaN,NaN,NaN
510,"Chikani, Umaymah",44:53.2,National University Singapore,12,6,Open,Run,10000,None,nan,...,NaN,NaN,NaN,2693.2,2693.2,NaN,NaN,NaN,NaN,NaN
765,"Low, Nicole",37:37.7,Wings Athletics Club,12,4,Open,Run,10000,None,N328A98,...,NaN,NaN,NaN,2257.7,2257.7,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12255,"Ranjan, Tharun",39:46.0,Nanyang Polytechnic,12,13,Open,Run,10000,None,nan,...,NaN,NaN,NaN,2386.0,2386.0,NaN,NaN,NaN,NaN,NaN
12271,"Mani, Manoj",40:19.8,Singapore Institute of Managem,12,9,Open,Run,10000,None,M323B00,...,NaN,NaN,NaN,2419.8,2419.8,NaN,NaN,NaN,NaN,NaN
12288,"Tan, Bernice",44:25.6,Lacticbuds,12,3,Open,Run,10000,None,B075C00,...,NaN,NaN,NaN,2665.6,2665.6,NaN,NaN,NaN,NaN,NaN
12780,"Ganesan, Kathiravan",40:39.6,Singapore Institute of Technol,12,14,Open,Run,10000,None,nan,...,NaN,NaN,NaN,2439.6,2439.6,NaN,NaN,NaN,NaN,NaN


In [3062]:
df

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,122.5224,124.32420,126.1260,114.0,114.00,8.5224,10.32420,12.1260,6.12,10.094905
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,NaN,NaN,NaN,1114.34,1114.34,NaN,NaN,NaN,NaN,NaN
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,13.3574,13.15295,12.9485,15.15,15.15,1.7926,1.99705,2.2015,1.52,16.151871
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,3.0184,2.97220,2.9260,3.2,3.20,0.1816,0.22780,0.2740,0.12,8.896104
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,146.0742,148.22235,150.3705,136.9,136.90,9.1742,11.32235,13.4705,6.31,9.406117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,St. Andrew's Junior College,03:43.06,SAJC,18.5,5.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,223.06,223.06,NaN,NaN,NaN,NaN,NaN
17329,Catholic Junior College,03:43.67,CJC,18.5,6.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,223.67,223.67,NaN,NaN,NaN,NaN,NaN
17330,Yishun Innova Junior College,03:45.03,YIJC,18.5,7.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,225.03,225.03,NaN,NaN,NaN,NaN,NaN
17331,National Junior College,03:47.40,NJC,18.5,8.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,227.4,227.40,NaN,NaN,NaN,NaN,NaN


In [3063]:
df['DOB_dt'].head(50)

0     1996-06-23 00:00:00
1                        
2     2003-02-23 00:00:00
3                        
4     1994-11-26 00:00:00
5     1994-11-26 00:00:00
6     1998-02-23 00:00:00
7     2000-10-14 00:00:00
8     2003-09-23 00:00:00
9     1996-09-20 00:00:00
10    1996-09-20 00:00:00
11    1994-07-03 00:00:00
12                       
13    2001-12-18 00:00:00
14    2003-09-23 00:00:00
15    2003-09-23 00:00:00
16    2003-09-23 00:00:00
17    1994-07-03 00:00:00
18    1996-02-26 00:00:00
19                       
20    1996-09-20 00:00:00
21    2004-01-06 00:00:00
22    2002-08-07 00:00:00
23    2002-08-07 00:00:00
24    2000-10-28 00:00:00
25    1997-01-12 00:00:00
26    1994-07-03 00:00:00
27    1996-02-26 00:00:00
28                       
29                       
30    1996-09-20 00:00:00
31    1996-09-20 00:00:00
32    1996-09-20 00:00:00
33    2006-01-22 00:00:00
34    1997-06-08 00:00:00
35    1997-06-08 00:00:00
36    2001-03-06 00:00:00
37                       
38    2004-0

In [3064]:
# Check that there are no empty DOB fields in dob_df
# Filter out empty DOB fields to build dictionary

#dob_df = df[((str(df['DOB_dt'])!='NaT')|(df['DOB_dt']!='None')|(df['DOB_dt'].notnull()))]
#dob_df = df.dropna(subset=['DOB_dt'])
dob_df = df[df['DOB_dt']!='']

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')

dob_df.to_csv('dob_df.csv', encoding='utf-8')

dob_df

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,122.5224,124.32420,126.1260,114.0,114.00,8.5224,10.32420,12.1260,6.12,10.094905
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,13.3574,13.15295,12.9485,15.15,15.15,1.7926,1.99705,2.2015,1.52,16.151871
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,146.0742,148.22235,150.3705,136.9,136.90,9.1742,11.32235,13.4705,6.31,9.406117
5,Romaine Soh,04:39.9,None,None,5.0,None,1500m,None,None,None,...,298.9110,303.30675,307.7025,279.9,279.90,19.0110,23.40675,27.8025,13.15,9.487289
6,Vanessa Lee Ying Zhuang,17:29.5,None,None,3.0,None,5000m,None,None,None,...,NaN,NaN,NaN,1049.5,1049.50,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12837,"TEO, MARK XAVIER",12.02m,Catholic High School,15,5,U18,Triple Jump,0,None,M391I10,...,13.3574,13.15295,12.9485,12.02,12.02,-1.3374,-1.13295,-0.9285,-1.61,-6.812179
12838,"SETHO, BEATRICE",9.76m,Nanyang Girls' High School,14,2,U15,Triple Jump,0,None,B332H11,...,11.6424,11.46420,11.2860,9.76,9.76,-1.8824,-1.70420,-1.5260,-2.12,-12.845118
12839,"Lau, Andrew",12.26m,Raffles Institution,15,5,U18,Triple Jump,0,None,A138H10,...,13.3574,13.15295,12.9485,12.26,12.26,-1.0974,-0.89295,-0.6885,-1.37,-5.051357
12840,"Yen, Ming Zhen",12.23m,National Junior College,17,4,U20,Triple Jump,0,None,M034B08,...,13.3574,13.15295,12.9485,12.23,12.23,-1.1274,-0.92295,-0.7185,-1.40,-5.271460


In [3065]:
# Create dictionary of NAME, DOB for everyone with a valid DOB
# Must map the name variation to a name from the df with all null DOB filtered out

dictionary_dob = dict(zip(dob_df['NAME'].str.casefold(), dob_df['DOB_dt']))

dictionary_dob

{'zubin percy muncherji': '1996-06-23 00:00:00',
 'gabriel lee': '2003-02-23 00:00:00',
 'romaine soh': '1994-11-26 00:00:00',
 'vanessa lee ying zhuang': '1998-02-23 00:00:00',
 'tia louise rozario': '2000-10-14 00:00:00',
 'elizabeth-ann tan': '2003-09-23 00:00:00',
 'shanti veronica pereira': '1996-09-20 00:00:00',
 'chen xiang ang': '1994-07-03 00:00:00',
 'tan daryl': '2001-12-18 00:00:00',
 'jun jie calvin quek': '1996-02-26 00:00:00',
 'jamie el-red ang': '2004-01-06 00:00:00',
 'marc brian louis': '2002-08-07 00:00:00',
 'rajan thiruben thana': '2000-10-28 00:00:00',
 'shaun goh': '1997-01-12 00:00:00',
 'laavina jaganith': '2006-01-22 00:00:00',
 'kerstin ong jing rong': '1997-06-08 00:00:00',
 'kampton kam': '2001-03-06 00:00:00',
 'joshua chua': '2004-02-25 00:00:00',
 'lucas le cong fun': '2006-01-11 00:00:00',
 'harry irfan curra': '2008-06-19 00:00:00',
 'yee chern clara lim': '2006-06-19 00:00:00',
 'ian koe': '1999-05-04 00:00:00',
 'raeshon loo wei ming': '2001-01-31 0

In [3070]:
# Create dictionary version with punctuation stripped out

translator = str.maketrans('', '', string.punctuation)
dob_df['clean_name'] = dob_df['NAME'].str.translate(translator)
dob_df['clean_name'] = dob_df['clean_name'].str.casefold()

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2522591649.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dob_df['clean_name'] = dob_df['NAME'].str.translate(translator)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2522591649.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dob_df['clean_name'] = dob_df['clean_name'].str.casefold()


In [3071]:
dob_df

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,clean_name
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,124.32420,126.1260,114.0,114.00,8.5224,10.32420,12.1260,6.12,10.094905,zubin percy muncherji
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,13.15295,12.9485,15.15,15.15,1.7926,1.99705,2.2015,1.52,16.151871,gabriel lee
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,148.22235,150.3705,136.9,136.90,9.1742,11.32235,13.4705,6.31,9.406117,romaine soh
5,Romaine Soh,04:39.9,None,None,5.0,None,1500m,None,None,None,...,303.30675,307.7025,279.9,279.90,19.0110,23.40675,27.8025,13.15,9.487289,romaine soh
6,Vanessa Lee Ying Zhuang,17:29.5,None,None,3.0,None,5000m,None,None,None,...,NaN,NaN,1049.5,1049.50,NaN,NaN,NaN,NaN,NaN,vanessa lee ying zhuang
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12837,"TEO, MARK XAVIER",12.02m,Catholic High School,15,5,U18,Triple Jump,0,None,M391I10,...,13.15295,12.9485,12.02,12.02,-1.3374,-1.13295,-0.9285,-1.61,-6.812179,teo mark xavier
12838,"SETHO, BEATRICE",9.76m,Nanyang Girls' High School,14,2,U15,Triple Jump,0,None,B332H11,...,11.46420,11.2860,9.76,9.76,-1.8824,-1.70420,-1.5260,-2.12,-12.845118,setho beatrice
12839,"Lau, Andrew",12.26m,Raffles Institution,15,5,U18,Triple Jump,0,None,A138H10,...,13.15295,12.9485,12.26,12.26,-1.0974,-0.89295,-0.6885,-1.37,-5.051357,lau andrew
12840,"Yen, Ming Zhen",12.23m,National Junior College,17,4,U20,Triple Jump,0,None,M034B08,...,13.15295,12.9485,12.23,12.23,-1.1274,-0.92295,-0.7185,-1.40,-5.271460,yen ming zhen


In [3072]:
dictionary_dob_clean_name = dict(zip(dob_df['clean_name'].str.casefold(), dob_df['DOB_dt']))


In [3073]:
dictionary_dob_clean_name

{'zubin percy muncherji': '1996-06-23 00:00:00',
 'gabriel lee': '2003-02-23 00:00:00',
 'romaine soh': '1994-11-26 00:00:00',
 'vanessa lee ying zhuang': '1998-02-23 00:00:00',
 'tia louise rozario': '2000-10-14 00:00:00',
 'elizabethann tan': '2003-09-23 00:00:00',
 'shanti veronica pereira': '1996-09-20 00:00:00',
 'chen xiang ang': '1994-07-03 00:00:00',
 'tan daryl': '2001-12-18 00:00:00',
 'jun jie calvin quek': '1996-02-26 00:00:00',
 'jamie elred ang': '2004-01-06 00:00:00',
 'marc brian louis': '2002-08-07 00:00:00',
 'rajan thiruben thana': '2000-10-28 00:00:00',
 'shaun goh': '1997-01-12 00:00:00',
 'laavina jaganith': '2006-01-22 00:00:00',
 'kerstin ong jing rong': '1997-06-08 00:00:00',
 'kampton kam': '2001-03-06 00:00:00',
 'joshua chua': '2004-02-25 00:00:00',
 'lucas le cong fun': '2006-01-11 00:00:00',
 'harry irfan curra': '2008-06-19 00:00:00',
 'yee chern clara lim': '2006-06-19 00:00:00',
 'ian koe': '1999-05-04 00:00:00',
 'raeshon loo wei ming': '2001-01-31 00:

In [3074]:
len(dictionary_dob)

5323

In [3075]:
dictionary_dob.get('chan, brayden wei jie')

'2007-08-25 00:00:00'

In [3076]:
'''
# Read a variation name list and corrections from CSVs

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Name Variations/')

names = pd.read_csv("junior_names.csv")

for index, row in names.iterrows():
        
    print(names.VARIATION, names.NAME)
    df['NAME_MAPPED'] = df['NAME'].replace(regex=rf"{row['VARIATION']}", value=f"{row['NAME']}")
'''

'\n# Read a variation name list and corrections from CSVs\n\nos.chdir(\'/Users/veesheenyuen/Desktop/DataScience/SAA/Name Variations/\')\n\nnames = pd.read_csv("junior_names.csv")\n\nfor index, row in names.iterrows():\n        \n    print(names.VARIATION, names.NAME)\n    df[\'NAME_MAPPED\'] = df[\'NAME\'].replace(regex=rf"{row[\'VARIATION\']}", value=f"{row[\'NAME\']}")\n'

In [3077]:
# Read name variations from GCS name lists bucket (Still in beta)

df['NAME'] = df['NAME'].str.replace('\xa0', '', regex=True)
df['NAME'] = df['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
df['NAME'] = df['NAME'].str.replace('\r', '', regex=True)
df['NAME'] = df['NAME'].str.replace('\n', '', regex=True)
df['NAME'] = df['NAME'].str.strip()

df['NAME'] = df['NAME'].str.casefold()  # everything lower case


# Read csv from GCS bucket

file_path = "gs://name_variations/name_variations.csv"
names = pd.read_csv(file_path,
                 sep=",",
                 storage_options={"token": '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json'})


# Clean and apply casefold to name variations

names['NAME'] = names['NAME'].str.replace('\xa0', '', regex=True)
names['NAME'] = names['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
names['NAME'] = names['NAME'].str.replace('\r', '', regex=True)
names['NAME'] = names['NAME'].str.replace('\n', '', regex=True)
names['NAME'] = names['NAME'].str.strip()
names['VARIATION'] = names['VARIATION'].str.replace('\xa0', '', regex=True)
names['VARIATION'] = names['VARIATION'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
names['VARIATION'] = names['VARIATION'].str.replace('\r', '', regex=True)
names['VARIATION'] = names['VARIATION'].str.replace('\n', '', regex=True)
names['VARIATION'] = names['VARIATION'].str.strip()

names['VARIATION'] = names['VARIATION'].str.casefold() # convert to lower case
names['NAME'] = names['NAME'].str.casefold()


# Iterate over dataframe and replace names

for index, row in names.iterrows():
        
    df['NAME'] = df['NAME'].replace(regex=rf"{row['VARIATION']}", value=f"{row['NAME']}")
    

    
df['NAME'] = df['NAME'].str.title()  # capitalize first letter


In [3078]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


df.to_csv("names_check.csv", encoding='utf-8')

In [3079]:
# Exclude foreigners from MALAYSIA, THAILAND etc.

#df_select = df[(df['TEAM']!='Malaysia') & (df['TEAM']!='THAILAND') & (df['TEAM']!='China') & (df['TEAM']!='South Korea') & (df['TEAM']!='Laos') & (df['TEAM']!='Philippines') & (df['TEAM']!='Piboonbumpen Thailand') & (df['TEAM']!='Chinese Taipei') & (df['TEAM']!='Gurkha Contingent') & (df['TEAM']!='Australia') & (df['TEAM']!='Piboonbumpen Thailand') & (df['TEAM']!='Hong Kong') & (df['TEAM']!='PERAK')] 

df_select = df[(df['TEAM']!='Malaysia')&(df['TEAM']!='THAILAND')&(df['TEAM']!='China')&(df['TEAM']!='Thailand') 
                       &(df['TEAM']!='South Korea')&(df['TEAM']!='Laos')&(df['TEAM']!='Myanmar') 
                       &(df['TEAM']!='Philippines')&(df['TEAM']!='Piboonbumpen Thailand') 
                       &(df['TEAM']!='Chinese Taipei')&(df['TEAM']!='Gurkha Contingent') 
                       &(df['TEAM']!='Australia')&(df['TEAM']!='Piboonbumpen Thailand') 
                       &(df['TEAM']!='Hong Kong')&(df['TEAM']!='PERAK')&(df['TEAM']!='Sri Lanka') 
                       &(df['TEAM']!='Indonesia')&(df['TEAM']!='THAILAND')&(df['TEAM']!='MALAYSIA') 
                       &(df['TEAM']!='PHILIPPINES') & (df['TEAM']!='SOUTH KOREA')&(df['TEAM']!='Waseda') 
                       &(df['TEAM']!='LAOS')&(df['TEAM']!='CHINESE TAIPEI')&(df['TEAM']!='Vietnam')
                       &(df['TEAM']!='INDIA')&(df['TEAM']!='Hong Kong, China')&(df['TEAM']!='AIC JAPAN')] 

In [3080]:
df_select

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,122.5224,124.32420,126.1260,114.0,114.00,8.5224,10.32420,12.1260,6.12,10.094905
1,"Low, Nicole Sui Xuan",18:34.34,None,None,6.0,None,5000m,None,None,None,...,NaN,NaN,NaN,1114.34,1114.34,NaN,NaN,NaN,NaN,NaN
2,Lee Gabriel Jin Yi,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,13.3574,13.15295,12.9485,15.15,15.15,1.7926,1.99705,2.2015,1.52,16.151871
3,"Tay, Esther",3.2,None,None,10.0,None,Pole Vault,None,None,None,...,3.0184,2.97220,2.9260,3.2,3.20,0.1816,0.22780,0.2740,0.12,8.896104
4,Romaine Soh Rui Min,02:16.90,None,None,5.0,None,800m,None,None,None,...,146.0742,148.22235,150.3705,136.9,136.90,9.1742,11.32235,13.4705,6.31,9.406117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,St. Andrew'S Junior College,03:43.06,SAJC,18.5,5.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,223.06,223.06,NaN,NaN,NaN,NaN,NaN
17329,Catholic Junior College,03:43.67,CJC,18.5,6.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,223.67,223.67,NaN,NaN,NaN,NaN,NaN
17330,Yishun Innova Junior College,03:45.03,YIJC,18.5,7.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,225.03,225.03,NaN,NaN,NaN,NaN,NaN
17331,National Junior College,03:47.40,NJC,18.5,8.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,227.4,227.40,NaN,NaN,NaN,NaN,NaN


In [3081]:
# Remove spurious SEED results 

#df_select=df_select[~((df_select['NAME']=='Tan, Kian Ye') & (df_select['MAPPED_EVENT']=='200m'))]
#df_select=df_select[~((df_select['NAME']=='LOH, BEAVERLY') & (df_select['MAPPED_EVENT']=='400m'))]
#df_select=df_select[~((df_select['NAME']=='Vaz, Luisa Marie') & (df_select['MAPPED_EVENT']=='High jump'))]


In [3082]:
df_select

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,122.5224,124.32420,126.1260,114.0,114.00,8.5224,10.32420,12.1260,6.12,10.094905
1,"Low, Nicole Sui Xuan",18:34.34,None,None,6.0,None,5000m,None,None,None,...,NaN,NaN,NaN,1114.34,1114.34,NaN,NaN,NaN,NaN,NaN
2,Lee Gabriel Jin Yi,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,13.3574,13.15295,12.9485,15.15,15.15,1.7926,1.99705,2.2015,1.52,16.151871
3,"Tay, Esther",3.2,None,None,10.0,None,Pole Vault,None,None,None,...,3.0184,2.97220,2.9260,3.2,3.20,0.1816,0.22780,0.2740,0.12,8.896104
4,Romaine Soh Rui Min,02:16.90,None,None,5.0,None,800m,None,None,None,...,146.0742,148.22235,150.3705,136.9,136.90,9.1742,11.32235,13.4705,6.31,9.406117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,St. Andrew'S Junior College,03:43.06,SAJC,18.5,5.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,223.06,223.06,NaN,NaN,NaN,NaN,NaN
17329,Catholic Junior College,03:43.67,CJC,18.5,6.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,223.67,223.67,NaN,NaN,NaN,NaN,NaN
17330,Yishun Innova Junior College,03:45.03,YIJC,18.5,7.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,225.03,225.03,NaN,NaN,NaN,NaN,NaN
17331,National Junior College,03:47.40,NJC,18.5,8.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,227.4,227.40,NaN,NaN,NaN,NaN,NaN


In [3083]:
'''
# Read list of foreigners

foreigners = pd.read_csv('/Users/veesheenyuen/Desktop/DataScience/SAA/MM/List of Foreigners.csv', encoding='latin-1')
'''


"\n# Read list of foreigners\n\nforeigners = pd.read_csv('/Users/veesheenyuen/Desktop/DataScience/SAA/MM/List of Foreigners.csv', encoding='latin-1')\n"

In [3084]:
# Read list of foreigners from GCS bucket

file_path = "gs://name_lists/List of Foreigners.csv"
foreigners = pd.read_csv(file_path,
                 sep=",",
                 encoding="unicode escape",
                 storage_options={"token": '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json'})


In [3085]:
foreigners

,LAST_NAME,FIRST_NAME
0,Aaryan,Greuter Christoph
1,Akahodani,Takayuki
2,Apondar,Audric
3,Brooks,Ruby
4,Brouwer,Cees
...,...,...
235,Kashama,Biwesa Daniel
236,ISMAIL,MUHAMMAD ZULFIQAR
237,Jayaganeson,Kirtisha
238,LIN,Yu Sian


In [3086]:
foreigners['V1'] = foreigners['LAST_NAME']+' '+foreigners['FIRST_NAME']
foreigners['V2'] = foreigners['FIRST_NAME']+' '+foreigners['LAST_NAME']
foreigners['V3'] = foreigners['LAST_NAME']+', '+foreigners['FIRST_NAME']
foreigners['V4'] = foreigners['FIRST_NAME']+' '+foreigners['LAST_NAME']

for1 = foreigners['V1'].dropna().tolist()
for2 = foreigners['V2'].dropna().tolist()
for3 = foreigners['V3'].dropna().tolist()
for4 = foreigners['V4'].dropna().tolist()

foreign_list = for1+for2+for3+for4 

foreign_list_casefold=[s.casefold() for s in foreign_list]

exclusions = foreign_list_casefold

no_foreigners_list = df_select.loc[~df['NAME'].str.casefold().isin(exclusions)]  # ~ means NOT IN. DROP spex carded athletes

In [3087]:
#rslt_df = df.loc[(df['RESULT_CONV'] < df['2pc']) & (df['AGE']<40) & (df['EVENT']!='Marathon')]

In [3088]:
#rslt_df = df.loc[(((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))&(df['RESULT_CONV'] >= df['5pc']) & (df['AGE']<40) & ((df['EVENT']!='Marathon')|(df['AGE']<60) & (df['EVENT']=='Marathon')))]

In [3089]:
no_foreigners_list

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,122.5224,124.32420,126.1260,114.0,114.00,8.5224,10.32420,12.1260,6.12,10.094905
1,"Low, Nicole Sui Xuan",18:34.34,None,None,6.0,None,5000m,None,None,None,...,NaN,NaN,NaN,1114.34,1114.34,NaN,NaN,NaN,NaN,NaN
2,Lee Gabriel Jin Yi,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,13.3574,13.15295,12.9485,15.15,15.15,1.7926,1.99705,2.2015,1.52,16.151871
3,"Tay, Esther",3.2,None,None,10.0,None,Pole Vault,None,None,None,...,3.0184,2.97220,2.9260,3.2,3.20,0.1816,0.22780,0.2740,0.12,8.896104
4,Romaine Soh Rui Min,02:16.90,None,None,5.0,None,800m,None,None,None,...,146.0742,148.22235,150.3705,136.9,136.90,9.1742,11.32235,13.4705,6.31,9.406117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,St. Andrew'S Junior College,03:43.06,SAJC,18.5,5.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,223.06,223.06,NaN,NaN,NaN,NaN,NaN
17329,Catholic Junior College,03:43.67,CJC,18.5,6.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,223.67,223.67,NaN,NaN,NaN,NaN,NaN
17330,Yishun Innova Junior College,03:45.03,YIJC,18.5,7.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,225.03,225.03,NaN,NaN,NaN,NaN,NaN
17331,National Junior College,03:47.40,NJC,18.5,8.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,227.4,227.40,NaN,NaN,NaN,NaN,NaN


In [3090]:
# Choose the best performing event for each athlete

#top_performers_clean = excluded_list.sort_values(['NAME','PERF_SCALAR'],ascending=False).groupby('NAME').head(1) # Choose top performing event per NAME


In [3091]:
# Choose the best result for each event participated by every athlete

top_performers_clean = no_foreigners_list.sort_values(['MAPPED_EVENT', 'NAME','PERF_SCALAR'],ascending=False).groupby(['MAPPED_EVENT', 'NAME']).head(1)


In [3092]:
top_performers_clean.reset_index(inplace=True)


In [3093]:
'''

# Process dates to extract age

# Map NSG divisions into age

mask = (top_performers_clean['DIVISION'].str.contains(r'A', na=False))
top_performers_clean.loc[mask, 'AGE'] = '18.5'

mask = (top_performers_clean['DIVISION'].str.contains(r'B', na=False))
top_performers_clean.loc[mask, 'AGE'] = '16'

mask = (top_performers_clean['DIVISION'].str.contains(r'C', na=False))
top_performers_clean.loc[mask, 'AGE'] = '13.5'

mask = (top_performers_clean['DIVISION'].str.contains(r'O', na=False))
top_performers_clean.loc[mask, 'AGE'] = '12'
'''

"\n\n# Process dates to extract age\n\n# Map NSG divisions into age\n\nmask = (top_performers_clean['DIVISION'].str.contains(r'A', na=False))\ntop_performers_clean.loc[mask, 'AGE'] = '18.5'\n\nmask = (top_performers_clean['DIVISION'].str.contains(r'B', na=False))\ntop_performers_clean.loc[mask, 'AGE'] = '16'\n\nmask = (top_performers_clean['DIVISION'].str.contains(r'C', na=False))\ntop_performers_clean.loc[mask, 'AGE'] = '13.5'\n\nmask = (top_performers_clean['DIVISION'].str.contains(r'O', na=False))\ntop_performers_clean.loc[mask, 'AGE'] = '12'\n"

In [3094]:
'''

def length(string):

    B = ''
    year = ''

    try:

        length = len(string)

        if length == 2:

            string = '19' + string

        elif length == 1:

            string = ''

        else:

            pass

        if string is not None or len(string) != 1:

            B = parser.parse(string, dayfirst=True)
                        
    except:

        pass

    return B


top_performers_clean['DOB_new'] = top_performers_clean['DOB'].apply(length)

'''


"\n\ndef length(string):\n\n    B = ''\n    year = ''\n\n    try:\n\n        length = len(string)\n\n        if length == 2:\n\n            string = '19' + string\n\n        elif length == 1:\n\n            string = ''\n\n        else:\n\n            pass\n\n        if string is not None or len(string) != 1:\n\n            B = parser.parse(string, dayfirst=True)\n                        \n    except:\n\n        pass\n\n    return B\n\n\ntop_performers_clean['DOB_new'] = top_performers_clean['DOB'].apply(length)\n\n"

In [3095]:
'''

top_performers_clean['DOB_new'] = pd.to_datetime(top_performers_clean['DOB_new'], errors='coerce')

top_performers_clean['year_extract'] = top_performers_clean['DOB_new'].dt.strftime('%Y')

top_performers_clean['year_extract'] = pd.to_numeric(top_performers_clean['year_extract'])

top_performers_clean['age_extract'] = 2025 - top_performers_clean['year_extract']
'''

"\n\ntop_performers_clean['DOB_new'] = pd.to_datetime(top_performers_clean['DOB_new'], errors='coerce')\n\ntop_performers_clean['year_extract'] = top_performers_clean['DOB_new'].dt.strftime('%Y')\n\ntop_performers_clean['year_extract'] = pd.to_numeric(top_performers_clean['year_extract'])\n\ntop_performers_clean['age_extract'] = 2025 - top_performers_clean['year_extract']\n"

In [3096]:
'''

def age(number):  # correct negative age numbers
    
    if number<0:
        
        number+=100
        
    return number


top_performers_clean['age_extract']=top_performers_clean['age_extract'].apply(age)
'''

"\n\ndef age(number):  # correct negative age numbers\n    \n    if number<0:\n        \n        number+=100\n        \n    return number\n\n\ntop_performers_clean['age_extract']=top_performers_clean['age_extract'].apply(age)\n"

In [3097]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


top_performers_clean.to_csv('AY_top_performers_prod.csv', encoding='utf-8')

In [3098]:
top_performers_clean

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,16545,Zhou Xuanyu,9.12,DHS,13.5,10.0,C,Triple Jump,None,None,...,11.6424,11.46420,11.2860,9.12,9.12,-2.5224,-2.34420,-2.1660,-2.76,-18.232323
1,11022,"Zhong, Chuhan",12.26,None,None,None,None,Triple Jump,None,None,...,11.6424,11.46420,11.2860,12.26,12.26,0.6176,0.79580,0.9740,0.38,8.198653
2,1076,"Zheng, Justin De",10.47m,National Junior College,14,12,U15,Triple Jump,0,None,...,13.3574,13.15295,12.9485,10.47,10.47,-2.8874,-2.68295,-2.4785,-3.16,-18.184153
3,12853,Zheng Justin De,10.29,NJC,13.5,12.0,C,Triple Jump,None,None,...,13.3574,13.15295,12.9485,10.29,10.29,-3.0674,-2.86295,-2.6585,-3.34,-19.504769
4,2326,"Zhao, Daniel",11.67m,Hwa Chong Institution,14,3,U15,Triple Jump,0,None,...,13.3574,13.15295,12.9485,11.67,11.67,-1.6874,-1.48295,-1.2785,-1.96,-9.380044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11373,3879,"., Shaik Isa",15.49,Club ZOOM,8,32,U9,Dash,80,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
11374,7899,"., Nur Elena",16.42,UNA,6,1,U7,Dash,80,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
11375,9907,"., Dharkshitha",08:35.6,Cedar Girls Secondary School,15,1,U18,Race Walk,1500,None,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
11376,3520,"., Cayden",06:05.1,North Vista,12,54,Open,Mile Run,1,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN


In [3099]:
# Choose best performance for each event

#tiered_performers = top_performers_clean.sort_values(['GENDER', 'MAPPED_EVENT', 'PERF_SCALAR'],ascending=False).groupby(['MAPPED_EVENT', 'NAME']).head(1)

tiered_performers = top_performers_clean


In [3100]:
tiered_performers

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,16545,Zhou Xuanyu,9.12,DHS,13.5,10.0,C,Triple Jump,None,None,...,11.6424,11.46420,11.2860,9.12,9.12,-2.5224,-2.34420,-2.1660,-2.76,-18.232323
1,11022,"Zhong, Chuhan",12.26,None,None,None,None,Triple Jump,None,None,...,11.6424,11.46420,11.2860,12.26,12.26,0.6176,0.79580,0.9740,0.38,8.198653
2,1076,"Zheng, Justin De",10.47m,National Junior College,14,12,U15,Triple Jump,0,None,...,13.3574,13.15295,12.9485,10.47,10.47,-2.8874,-2.68295,-2.4785,-3.16,-18.184153
3,12853,Zheng Justin De,10.29,NJC,13.5,12.0,C,Triple Jump,None,None,...,13.3574,13.15295,12.9485,10.29,10.29,-3.0674,-2.86295,-2.6585,-3.34,-19.504769
4,2326,"Zhao, Daniel",11.67m,Hwa Chong Institution,14,3,U15,Triple Jump,0,None,...,13.3574,13.15295,12.9485,11.67,11.67,-1.6874,-1.48295,-1.2785,-1.96,-9.380044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11373,3879,"., Shaik Isa",15.49,Club ZOOM,8,32,U9,Dash,80,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
11374,7899,"., Nur Elena",16.42,UNA,6,1,U7,Dash,80,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
11375,9907,"., Dharkshitha",08:35.6,Cedar Girls Secondary School,15,1,U18,Race Walk,1500,None,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
11376,3520,"., Cayden",06:05.1,North Vista,12,54,Open,Mile Run,1,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN


In [3101]:
# Identify Tier 1/2/3 performers

#top_performers_clean['TIER'] = np.where((top_performers_clean['Delta_Benchmark']>=0), 'Tier 1',    
#                                np.where(((top_performers_clean['Delta_Benchmark']<0) & (top_performers_clean['Delta2']>=0)), 'Tier2',
#                                np.where(((top_performers_clean['Delta2']<0) & (top_performers_clean['Delta3.5']>=0)), 'Tier3', ' ')))


tiered_performers['TIER'] = np.where((tiered_performers['Delta_Benchmark']>=0), 'Tier 1',    
                                np.where(((tiered_performers['Delta_Benchmark']<0) & (tiered_performers['Delta2']>=0)), 'Tier 2',
                                np.where(((tiered_performers['Delta2']<0) & (tiered_performers['Delta3.5']>=0)), 'Tier 3', 
                                np.where(((tiered_performers['Delta3.5']<0) & (tiered_performers['Delta5']>=0)), 'Tier 4', ' '))))



In [3102]:
# Drop rows without a SEAG benchmark

final_df = tiered_performers[tiered_performers['BENCHMARK'].notna()]


In [3103]:
final_df

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER
0,16545,Zhou Xuanyu,9.12,DHS,13.5,10.0,C,Triple Jump,None,None,...,11.46420,11.2860,9.12,9.12,-2.5224,-2.34420,-2.1660,-2.76,-18.232323,
1,11022,"Zhong, Chuhan",12.26,None,None,None,None,Triple Jump,None,None,...,11.46420,11.2860,12.26,12.26,0.6176,0.79580,0.9740,0.38,8.198653,Tier 1
2,1076,"Zheng, Justin De",10.47m,National Junior College,14,12,U15,Triple Jump,0,None,...,13.15295,12.9485,10.47,10.47,-2.8874,-2.68295,-2.4785,-3.16,-18.184153,
3,12853,Zheng Justin De,10.29,NJC,13.5,12.0,C,Triple Jump,None,None,...,13.15295,12.9485,10.29,10.29,-3.0674,-2.86295,-2.6585,-3.34,-19.504769,
4,2326,"Zhao, Daniel",11.67m,Hwa Chong Institution,14,3,U15,Triple Jump,0,None,...,13.15295,12.9485,11.67,11.67,-1.6874,-1.48295,-1.2785,-1.96,-9.380044,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11373,3879,"., Shaik Isa",15.49,Club ZOOM,8,32,U9,Dash,80,,...,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,
11374,7899,"., Nur Elena",16.42,UNA,6,1,U7,Dash,80,,...,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,
11375,9907,"., Dharkshitha",08:35.6,Cedar Girls Secondary School,15,1,U18,Race Walk,1500,None,...,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,
11376,3520,"., Cayden",06:05.1,North Vista,12,54,Open,Mile Run,1,,...,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,


In [3104]:
# Choose rows with mapped event

final_df=final_df[(final_df['MAPPED_EVENT']!=' ') & (final_df['TIER']!=' ')]

In [3105]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


final_df.to_csv('AY_tiered_performers.csv', encoding='utf-8')

In [3106]:
final_df

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER
1,11022,"Zhong, Chuhan",12.26,None,None,None,None,Triple Jump,None,None,...,11.46420,11.2860,12.26,12.26,0.6176,0.79580,0.9740,0.38,8.198653,Tier 1
47,12316,Tse Teng Tan,11.59,None,None,None,None,Triple Jump,None,None,...,11.46420,11.2860,11.59,11.59,-0.0524,0.12580,0.3040,-0.29,2.558923,Tier 3
57,6870,"Tang Kai Sheng, Cayman",14.17m,VICTORIA SCHOOL,16,1,U18,Triple Jump,0,None,...,13.15295,12.9485,14.17,14.17,0.8126,1.01705,1.2215,0.54,8.961849,Tier 1
61,11299,"Tan, Tse Teng",11.77m,Hwa Chong Alumni Association,12,2,Open,Triple Jump,0,None,...,11.46420,11.2860,11.77,11.77,0.1276,0.30580,0.4840,-0.11,4.074074,Tier 2
79,9726,Tan Shou Yi Rei,14.99m,Raffles Institution JC,17,1,U20,Triple Jump,0,None,...,13.15295,12.9485,14.99,14.99,1.6326,1.83705,2.0415,1.36,14.977990,Tier 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8463,1883,"Bin Zainal, Muhammad Fazrey",11.32,Singapore Institute of Technol,12,17,Open,Dash,100,,...,11.22975,11.3925,11.32,11.32,-0.2530,-0.09025,0.0725,-0.47,0.668203,Tier 4
8507,1370,"Ashok, Tharun",10.81,Ngee Ann Polytechnic,12,4,Open,Dash,100,,...,11.22975,11.3925,10.81,10.81,0.2570,0.41975,0.5825,0.04,5.368664,Tier 1
8534,889,"Ang, Jamie El-Red",11.09,Oldham Athletics,12,19,Open,Dash,100,None,...,11.22975,11.3925,11.09,11.09,-0.0230,0.13975,0.3025,-0.24,2.788018,Tier 3
8557,3341,Aidan Michael Soh,11.16,INDIVIDUAL,18,None,None,100m,None,None,...,11.22975,11.3925,11.16,11.16,-0.0930,0.06975,0.2325,-0.31,2.142857,Tier 3


In [3107]:
#os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


#final_selection.to_csv('AY_final_selection.csv', encoding='utf-8')

In [3108]:
# Get list of names with no DOB

mask = ((final_df['DOB']=='None')|(final_df['DOB']=='nan'))

#mask = ((final_df['DOB'].isnull()))
#mask = final_df['DOB_new']=='NaT'

missing_names = final_df.loc[mask]

name_list = missing_names['NAME'].str.casefold().to_list()


In [3109]:
len(name_list)

120

In [3110]:
# Names with no DOB

missing_names

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER
47,12316,Tse Teng Tan,11.59,None,None,None,None,Triple Jump,None,None,...,11.46420,11.2860,11.59,11.59,-0.0524,0.12580,0.3040,-0.29,2.558923,Tier 3
100,804,"Saw, Xiang Yu",13.92m,Singapore Institute of Managem,12,2,Open,Triple Jump,0,None,...,13.15295,12.9485,13.92,13.92,0.5626,0.76705,0.9715,0.29,7.127660,Tier 1
196,17015,"Lau, Jia Hern",13.63,RI,18.5,3.0,A,Triple Jump,None,None,...,13.15295,12.9485,13.63,13.63,0.2726,0.47705,0.6815,0.00,5.000000,Tier 1
243,17016,"Fu, Song",13.52,EJC,18.5,4.0,A,Triple Jump,None,None,...,13.15295,12.9485,13.52,13.52,0.1626,0.36705,0.5715,-0.11,4.192957,Tier 2
276,17018,Chang I Teng,13.01,DHS,18.5,6.0,A,Triple Jump,None,None,...,13.15295,12.9485,13.01,13.01,-0.3474,-0.14295,0.0615,-0.62,0.451211,Tier 4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8306,17173,"Chua, Garrett",00:10.90,RI,18.5,2.0,A,100m,None,None,...,11.22975,11.3925,10.9,10.90,0.1670,0.32975,0.4925,-0.05,4.539171,Tier 2
8343,15836,Chloe Sarahena Tan Hui En,00:12.85,EJC,18.5,6.0,A,100m,None,None,...,12.67875,12.8625,12.85,12.85,-0.3550,-0.17125,0.0125,-0.60,0.102041,Tier 4
8344,17152,Chloe Chee En-Ya,00:12.44,ACS(I),18.5,4.0,A,100m,None,None,...,12.67875,12.8625,12.44,12.44,0.0550,0.23875,0.4225,-0.19,3.448980,Tier 2
8449,10792,Bryan Ng,10.93,None,None,2,None,100m,None,None,...,11.22975,11.3925,10.93,10.93,0.1370,0.29975,0.4625,-0.08,4.262673,Tier 2


In [3111]:
final_df['DOB_dt']

1       2005-10-18 00:00:00
47                         
57      2009-07-15 00:00:00
61      2002-11-28 00:00:00
79      2008-12-05 00:00:00
               ...         
8463    1997-09-04 00:00:00
8507    2005-07-02 00:00:00
8534    2004-01-06 00:00:00
8557    2007-06-19 00:00:00
8569    2007-07-06 00:00:00
Name: DOB_dt, Length: 461, dtype: object

In [3112]:
# Iterate over df to fill in missing DOBs from dictionary created above
count=0

for index, row in final_df.iterrows():
         
   # if str(row['DOB_new']) == 'NaT':
   # if ((final_df['DOB']=='None') or (final_df['DOB']=='nan')):
    if row['DOB_dt']=='':
            
        name = str(row['NAME']).casefold()
        
        print(name, dictionary_dob.get(name))  
        
        final_df.loc[index, 'DOB_missing'] = dictionary_dob.get(name)
        
        count+=1
        
print('COUNT', count)

tse teng tan None
saw, xiang yu 2004-10-31 00:00:00
medina, andrew george 2002-03-22 00:00:00
lau, jia hern 2008-09-03 00:00:00
fu, song 2007-07-13 00:00:00
chang i teng None
ng, gabriel 2008-07-24 00:00:00
khalel bin zaid None
foo ming zhe, sean None
cheong, joseph 2007-01-27 00:00:00
toh jun xi, tedd 2002-06-19 00:00:00
han lin feng None
chua, garrett 2008-02-28 00:00:00
ryan tan wei zhong None
lam, kristel 2010-12-22 00:00:00
kwa eu han 2008-06-19 00:00:00
dexton phua jun xiang None
wong, jin rou 2006-08-01 00:00:00
tan shou yi rei 2007-06-19 00:00:00
tan, emma yi min 2007-04-18 00:00:00
tan jun jie None
jaden chew 2008-06-19 00:00:00
wee, nicholas 2006-06-02 00:00:00
tay, kai bin 2001-01-27 00:00:00
tan wei yee None
su, emma 2007-04-23 00:00:00
solaimuthu kausik None
shanice tan haridas None
saicharan ganeshkumar None
megan puah None
lim tze rong, oliver None
kuah yong chuan, ezekiel None
kimberly chew pei min None
jeffrey, deianira 2010-09-30 00:00:00
jeevan joshua jerom None
henr

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2032803433.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.loc[index, 'DOB_missing'] = dictionary_dob.get(name)


In [3113]:
final_df

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_missing
1,11022,"Zhong, Chuhan",12.26,None,None,None,None,Triple Jump,None,None,...,11.2860,12.26,12.26,0.6176,0.79580,0.9740,0.38,8.198653,Tier 1,NaN
47,12316,Tse Teng Tan,11.59,None,None,None,None,Triple Jump,None,None,...,11.2860,11.59,11.59,-0.0524,0.12580,0.3040,-0.29,2.558923,Tier 3,None
57,6870,"Tang Kai Sheng, Cayman",14.17m,VICTORIA SCHOOL,16,1,U18,Triple Jump,0,None,...,12.9485,14.17,14.17,0.8126,1.01705,1.2215,0.54,8.961849,Tier 1,NaN
61,11299,"Tan, Tse Teng",11.77m,Hwa Chong Alumni Association,12,2,Open,Triple Jump,0,None,...,11.2860,11.77,11.77,0.1276,0.30580,0.4840,-0.11,4.074074,Tier 2,NaN
79,9726,Tan Shou Yi Rei,14.99m,Raffles Institution JC,17,1,U20,Triple Jump,0,None,...,12.9485,14.99,14.99,1.6326,1.83705,2.0415,1.36,14.977990,Tier 1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8463,1883,"Bin Zainal, Muhammad Fazrey",11.32,Singapore Institute of Technol,12,17,Open,Dash,100,,...,11.3925,11.32,11.32,-0.2530,-0.09025,0.0725,-0.47,0.668203,Tier 4,NaN
8507,1370,"Ashok, Tharun",10.81,Ngee Ann Polytechnic,12,4,Open,Dash,100,,...,11.3925,10.81,10.81,0.2570,0.41975,0.5825,0.04,5.368664,Tier 1,NaN
8534,889,"Ang, Jamie El-Red",11.09,Oldham Athletics,12,19,Open,Dash,100,None,...,11.3925,11.09,11.09,-0.0230,0.13975,0.3025,-0.24,2.788018,Tier 3,NaN
8557,3341,Aidan Michael Soh,11.16,INDIVIDUAL,18,None,None,100m,None,None,...,11.3925,11.16,11.16,-0.0930,0.06975,0.2325,-0.31,2.142857,Tier 3,NaN


In [3114]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')

final_df.to_csv('check_missing_dob_map.csv', encoding='utf-8')

In [3115]:
# Consolidate all the DOB fields into one column

condition1 = final_df['DOB_dt']!=''
#condition2=((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))
#condition3=df['SEED_CONV']<df['RESULT_CONV']
#condition4=~((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))


final_df['DOB_final'] = final_df['DOB_dt'].where((condition1), final_df['DOB_missing'].values)


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2872990833.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['DOB_final'] = final_df['DOB_dt'].where((condition1), final_df['DOB_missing'].values)


In [3116]:
final_df

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_missing,DOB_final
1,11022,"Zhong, Chuhan",12.26,None,None,None,None,Triple Jump,None,None,...,12.26,12.26,0.6176,0.79580,0.9740,0.38,8.198653,Tier 1,NaN,2005-10-18 00:00:00
47,12316,Tse Teng Tan,11.59,None,None,None,None,Triple Jump,None,None,...,11.59,11.59,-0.0524,0.12580,0.3040,-0.29,2.558923,Tier 3,None,None
57,6870,"Tang Kai Sheng, Cayman",14.17m,VICTORIA SCHOOL,16,1,U18,Triple Jump,0,None,...,14.17,14.17,0.8126,1.01705,1.2215,0.54,8.961849,Tier 1,NaN,2009-07-15 00:00:00
61,11299,"Tan, Tse Teng",11.77m,Hwa Chong Alumni Association,12,2,Open,Triple Jump,0,None,...,11.77,11.77,0.1276,0.30580,0.4840,-0.11,4.074074,Tier 2,NaN,2002-11-28 00:00:00
79,9726,Tan Shou Yi Rei,14.99m,Raffles Institution JC,17,1,U20,Triple Jump,0,None,...,14.99,14.99,1.6326,1.83705,2.0415,1.36,14.977990,Tier 1,NaN,2008-12-05 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8463,1883,"Bin Zainal, Muhammad Fazrey",11.32,Singapore Institute of Technol,12,17,Open,Dash,100,,...,11.32,11.32,-0.2530,-0.09025,0.0725,-0.47,0.668203,Tier 4,NaN,1997-09-04 00:00:00
8507,1370,"Ashok, Tharun",10.81,Ngee Ann Polytechnic,12,4,Open,Dash,100,,...,10.81,10.81,0.2570,0.41975,0.5825,0.04,5.368664,Tier 1,NaN,2005-07-02 00:00:00
8534,889,"Ang, Jamie El-Red",11.09,Oldham Athletics,12,19,Open,Dash,100,None,...,11.09,11.09,-0.0230,0.13975,0.3025,-0.24,2.788018,Tier 3,NaN,2004-01-06 00:00:00
8557,3341,Aidan Michael Soh,11.16,INDIVIDUAL,18,None,None,100m,None,None,...,11.16,11.16,-0.0930,0.06975,0.2325,-0.31,2.142857,Tier 3,NaN,2007-06-19 00:00:00


In [3117]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')

final_df.to_csv('dob_map_final.csv', encoding='utf-8')

In [3118]:
# All the remaining names where the dictionary has no match

null = final_df[(final_df['DOB_final'].isnull())]

null

/Users/veesheenyuen/.pyenv/versions/3.10.6/envs/SAA/lib/python3.10/site-packages/IPython/core/displayhook.py:275: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_missing,DOB_final
47,12316,Tse Teng Tan,11.59,None,None,None,None,Triple Jump,None,None,...,11.59,11.59,-0.0524,0.12580,0.3040,-0.29,2.558923,Tier 3,None,None
276,17018,Chang I Teng,13.01,DHS,18.5,6.0,A,Triple Jump,None,None,...,13.01,13.01,-0.3474,-0.14295,0.0615,-0.62,0.451211,Tier 4,None,None
513,1082,Khalel Bin Zaid,4.05,RI,18.5,2.0,A,Pole Vault,None,None,...,4.05,4.05,0.1006,0.16105,0.2215,0.02,5.496278,Tier 1,None,None
533,14181,"Foo Ming Zhe, Sean",3.85,RI,16,1.0,B,Pole Vault,None,None,...,3.85,3.85,-0.0994,-0.03895,0.0215,-0.18,0.533499,Tier 4,None,None
1329,7407,Han Lin Feng,7.08,None,None,None,None,Long Jump,None,None,...,7.08,7.08,0.2788,0.38290,0.4870,0.14,7.017291,Tier 1,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8085,5081,Joshua Shyen Lee,11.18,None,None,2,None,100m,None,None,...,11.18,11.18,-0.1130,0.04975,0.2125,-0.33,1.958525,Tier 3,None,None
8091,17177,John Andre Chua Yi Kai,00:11.37,CJC,18.5,6.0,A,100m,None,None,...,11.37,11.37,-0.3030,-0.14025,0.0225,-0.52,0.207373,Tier 4,None,None
8343,15836,Chloe Sarahena Tan Hui En,00:12.85,EJC,18.5,6.0,A,100m,None,None,...,12.85,12.85,-0.3550,-0.17125,0.0125,-0.60,0.102041,Tier 4,None,None
8449,10792,Bryan Ng,10.93,None,None,2,None,100m,None,None,...,10.93,10.93,0.1370,0.29975,0.4625,-0.08,4.262673,Tier 2,None,None


In [3119]:
# Use clean name without punctuation and dictionary_dob_clean_name to lookup the rest

# Create dictionary version with punctuation stripped out

translator = str.maketrans('', '', string.punctuation)
null['clean_name'] = null['NAME'].str.translate(translator)
null['clean_name'] = null['clean_name'].str.casefold()

null

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/4056426607.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null['clean_name'] = null['NAME'].str.translate(translator)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/4056426607.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null['clean_name'] = null['clean_name'].str.casefold()


,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_missing,DOB_final,clean_name
47,12316,Tse Teng Tan,11.59,None,None,None,None,Triple Jump,None,None,...,11.59,-0.0524,0.12580,0.3040,-0.29,2.558923,Tier 3,None,None,tse teng tan
276,17018,Chang I Teng,13.01,DHS,18.5,6.0,A,Triple Jump,None,None,...,13.01,-0.3474,-0.14295,0.0615,-0.62,0.451211,Tier 4,None,None,chang i teng
513,1082,Khalel Bin Zaid,4.05,RI,18.5,2.0,A,Pole Vault,None,None,...,4.05,0.1006,0.16105,0.2215,0.02,5.496278,Tier 1,None,None,khalel bin zaid
533,14181,"Foo Ming Zhe, Sean",3.85,RI,16,1.0,B,Pole Vault,None,None,...,3.85,-0.0994,-0.03895,0.0215,-0.18,0.533499,Tier 4,None,None,foo ming zhe sean
1329,7407,Han Lin Feng,7.08,None,None,None,None,Long Jump,None,None,...,7.08,0.2788,0.38290,0.4870,0.14,7.017291,Tier 1,None,None,han lin feng
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8085,5081,Joshua Shyen Lee,11.18,None,None,2,None,100m,None,None,...,11.18,-0.1130,0.04975,0.2125,-0.33,1.958525,Tier 3,None,None,joshua shyen lee
8091,17177,John Andre Chua Yi Kai,00:11.37,CJC,18.5,6.0,A,100m,None,None,...,11.37,-0.3030,-0.14025,0.0225,-0.52,0.207373,Tier 4,None,None,john andre chua yi kai
8343,15836,Chloe Sarahena Tan Hui En,00:12.85,EJC,18.5,6.0,A,100m,None,None,...,12.85,-0.3550,-0.17125,0.0125,-0.60,0.102041,Tier 4,None,None,chloe sarahena tan hui en
8449,10792,Bryan Ng,10.93,None,None,2,None,100m,None,None,...,10.93,0.1370,0.29975,0.4625,-0.08,4.262673,Tier 2,None,None,bryan ng


In [3120]:
dictionary_dob_clean_name

{'zubin percy muncherji': '1996-06-23 00:00:00',
 'gabriel lee': '2003-02-23 00:00:00',
 'romaine soh': '1994-11-26 00:00:00',
 'vanessa lee ying zhuang': '1998-02-23 00:00:00',
 'tia louise rozario': '2000-10-14 00:00:00',
 'elizabethann tan': '2003-09-23 00:00:00',
 'shanti veronica pereira': '1996-09-20 00:00:00',
 'chen xiang ang': '1994-07-03 00:00:00',
 'tan daryl': '2001-12-18 00:00:00',
 'jun jie calvin quek': '1996-02-26 00:00:00',
 'jamie elred ang': '2004-01-06 00:00:00',
 'marc brian louis': '2002-08-07 00:00:00',
 'rajan thiruben thana': '2000-10-28 00:00:00',
 'shaun goh': '1997-01-12 00:00:00',
 'laavina jaganith': '2006-01-22 00:00:00',
 'kerstin ong jing rong': '1997-06-08 00:00:00',
 'kampton kam': '2001-03-06 00:00:00',
 'joshua chua': '2004-02-25 00:00:00',
 'lucas le cong fun': '2006-01-11 00:00:00',
 'harry irfan curra': '2008-06-19 00:00:00',
 'yee chern clara lim': '2006-06-19 00:00:00',
 'ian koe': '1999-05-04 00:00:00',
 'raeshon loo wei ming': '2001-01-31 00:

In [3121]:
# Iterate over df to fill in missing DOBs from clean name without punctuation dictionary created above
count=0

for index, row in null.iterrows():
         
   # if str(row['DOB_new']) == 'NaT':
   # if ((final_df['DOB']=='None') or (final_df['DOB']=='nan')):
            
    name = row['clean_name']
        
    print(name, dictionary_dob_clean_name.get(name))  
        
    null.loc[index, 'DOB_found'] = dictionary_dob_clean_name.get(name)
        
    count+=1
        
print('COUNT', count)

tse teng tan None
chang i teng 2007-05-04 00:00:00
khalel bin zaid None
foo ming zhe sean None
han lin feng None
ryan tan wei zhong None
dexton phua jun xiang None
tan jun jie 2007-02-07 00:00:00
tan wei yee 2007-01-20 00:00:00
solaimuthu kausik 2007-09-24 00:00:00
shanice tan haridas None
saicharan ganeshkumar None
megan puah None
lim tze rong oliver None
kuah yong chuan ezekiel 2009-02-12 00:00:00
kimberly chew pei min None
jeevan joshua jerom None
henry zhao shuheng None
foo si qing faith 2007-06-28 00:00:00
ethan lee jinxian None
cai zeyang 2008-03-19 00:00:00
abdul hadi bin abdullah None
toh xinyi isabel None
caelyn chew kai ling None
toh xinyi isabel None
sean gerard yoo tze you None
henry zhao shuheng None
emily anne choi yi siu None
curran harry irfan irfan None
bharadwaj akash vijay None
alexis teo pei qi None
aidan soon qi zhe None
aidan michael soh zi ren None
wang qiyue 2009-08-12 00:00:00
tong zong wei 2007-04-16 00:00:00
thaddaeus lee sheng en None
soh hwei en rachel su h

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/4267687476.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null.loc[index, 'DOB_found'] = dictionary_dob_clean_name.get(name)


In [3122]:
null

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_missing,DOB_final,clean_name,DOB_found
47,12316,Tse Teng Tan,11.59,None,None,None,None,Triple Jump,None,None,...,-0.0524,0.12580,0.3040,-0.29,2.558923,Tier 3,None,None,tse teng tan,None
276,17018,Chang I Teng,13.01,DHS,18.5,6.0,A,Triple Jump,None,None,...,-0.3474,-0.14295,0.0615,-0.62,0.451211,Tier 4,None,None,chang i teng,2007-05-04 00:00:00
513,1082,Khalel Bin Zaid,4.05,RI,18.5,2.0,A,Pole Vault,None,None,...,0.1006,0.16105,0.2215,0.02,5.496278,Tier 1,None,None,khalel bin zaid,None
533,14181,"Foo Ming Zhe, Sean",3.85,RI,16,1.0,B,Pole Vault,None,None,...,-0.0994,-0.03895,0.0215,-0.18,0.533499,Tier 4,None,None,foo ming zhe sean,None
1329,7407,Han Lin Feng,7.08,None,None,None,None,Long Jump,None,None,...,0.2788,0.38290,0.4870,0.14,7.017291,Tier 1,None,None,han lin feng,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8085,5081,Joshua Shyen Lee,11.18,None,None,2,None,100m,None,None,...,-0.1130,0.04975,0.2125,-0.33,1.958525,Tier 3,None,None,joshua shyen lee,None
8091,17177,John Andre Chua Yi Kai,00:11.37,CJC,18.5,6.0,A,100m,None,None,...,-0.3030,-0.14025,0.0225,-0.52,0.207373,Tier 4,None,None,john andre chua yi kai,None
8343,15836,Chloe Sarahena Tan Hui En,00:12.85,EJC,18.5,6.0,A,100m,None,None,...,-0.3550,-0.17125,0.0125,-0.60,0.102041,Tier 4,None,None,chloe sarahena tan hui en,None
8449,10792,Bryan Ng,10.93,None,None,2,None,100m,None,None,...,0.1370,0.29975,0.4625,-0.08,4.262673,Tier 2,None,None,bryan ng,None


In [3123]:
# Split name elements into a list


null['name_to_list'] = null['clean_name'].str.split(' ')

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3834781152.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null['name_to_list'] = null['clean_name'].str.split(' ')


In [3124]:
null = null.reset_index(drop=True)

null

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_missing,DOB_final,clean_name,DOB_found,name_to_list
0,12316,Tse Teng Tan,11.59,None,None,None,None,Triple Jump,None,None,...,0.12580,0.3040,-0.29,2.558923,Tier 3,None,None,tse teng tan,None,"[tse, teng, tan]"
1,17018,Chang I Teng,13.01,DHS,18.5,6.0,A,Triple Jump,None,None,...,-0.14295,0.0615,-0.62,0.451211,Tier 4,None,None,chang i teng,2007-05-04 00:00:00,"[chang, i, teng]"
2,1082,Khalel Bin Zaid,4.05,RI,18.5,2.0,A,Pole Vault,None,None,...,0.16105,0.2215,0.02,5.496278,Tier 1,None,None,khalel bin zaid,None,"[khalel, bin, zaid]"
3,14181,"Foo Ming Zhe, Sean",3.85,RI,16,1.0,B,Pole Vault,None,None,...,-0.03895,0.0215,-0.18,0.533499,Tier 4,None,None,foo ming zhe sean,None,"[foo, ming, zhe, sean]"
4,7407,Han Lin Feng,7.08,None,None,None,None,Long Jump,None,None,...,0.38290,0.4870,0.14,7.017291,Tier 1,None,None,han lin feng,None,"[han, lin, feng]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,5081,Joshua Shyen Lee,11.18,None,None,2,None,100m,None,None,...,0.04975,0.2125,-0.33,1.958525,Tier 3,None,None,joshua shyen lee,None,"[joshua, shyen, lee]"
61,17177,John Andre Chua Yi Kai,00:11.37,CJC,18.5,6.0,A,100m,None,None,...,-0.14025,0.0225,-0.52,0.207373,Tier 4,None,None,john andre chua yi kai,None,"[john, andre, chua, yi, kai]"
62,15836,Chloe Sarahena Tan Hui En,00:12.85,EJC,18.5,6.0,A,100m,None,None,...,-0.17125,0.0125,-0.60,0.102041,Tier 4,None,None,chloe sarahena tan hui en,None,"[chloe, sarahena, tan, hui, en]"
63,10792,Bryan Ng,10.93,None,None,2,None,100m,None,None,...,0.29975,0.4625,-0.08,4.262673,Tier 2,None,None,bryan ng,None,"[bryan, ng]"


In [3129]:
# Determine all permutations of name

for index, row in null.iterrows():
         
   # if str(row['DOB_new']) == 'NaT':
   # if ((final_df['DOB']=='None') or (final_df['DOB']=='nan')):
            
    name_components = row['name_to_list']
        
    permutations_list = []
    dob_list=[]
    string=''
    
    
    for p in permutations(name_components):
        
        string = " ".join(p)
        
        permutations_list.append(string)

        dob = dictionary_dob_clean_name.get(string)
        
        if dob is not None:
            dob_list.append(dob)

    
  #  null.at[index, 'permutations'] = permutations_list
    null.at[index, 'dob_list'] = dob_list
   # print(name, dictionary_dob_clean_name.get(name))  
        
    #null.loc[index, 'DOB_found'] = dictionary_dob_clean_name.get(name)


In [3130]:
null

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,Delta_Benchmark,PERF_SCALAR,TIER,DOB_missing,DOB_final,clean_name,DOB_found,name_to_list,permutations,dob_list
0,12316,Tse Teng Tan,11.59,None,None,None,None,Triple Jump,None,None,...,-0.29,2.558923,Tier 3,None,None,tse teng tan,None,"[tse, teng, tan]",nan,[2002-11-28 00:00:00]
1,17018,Chang I Teng,13.01,DHS,18.5,6.0,A,Triple Jump,None,None,...,-0.62,0.451211,Tier 4,None,None,chang i teng,2007-05-04 00:00:00,"[chang, i, teng]",nan,[2007-05-04 00:00:00]
2,1082,Khalel Bin Zaid,4.05,RI,18.5,2.0,A,Pole Vault,None,None,...,0.02,5.496278,Tier 1,None,None,khalel bin zaid,None,"[khalel, bin, zaid]",nan,[]
3,14181,"Foo Ming Zhe, Sean",3.85,RI,16,1.0,B,Pole Vault,None,None,...,-0.18,0.533499,Tier 4,None,None,foo ming zhe sean,None,"[foo, ming, zhe, sean]",nan,[]
4,7407,Han Lin Feng,7.08,None,None,None,None,Long Jump,None,None,...,0.14,7.017291,Tier 1,None,None,han lin feng,None,"[han, lin, feng]",nan,[2004-03-08 00:00:00]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,5081,Joshua Shyen Lee,11.18,None,None,2,None,100m,None,None,...,-0.33,1.958525,Tier 3,None,None,joshua shyen lee,None,"[joshua, shyen, lee]",nan,"[2008-09-12 00:00:00, 2008-09-12 00:00:00]"
61,17177,John Andre Chua Yi Kai,00:11.37,CJC,18.5,6.0,A,100m,None,None,...,-0.52,0.207373,Tier 4,None,None,john andre chua yi kai,None,"[john, andre, chua, yi, kai]",nan,[]
62,15836,Chloe Sarahena Tan Hui En,00:12.85,EJC,18.5,6.0,A,100m,None,None,...,-0.60,0.102041,Tier 4,None,None,chloe sarahena tan hui en,None,"[chloe, sarahena, tan, hui, en]",nan,[]
63,10792,Bryan Ng,10.93,None,None,2,None,100m,None,None,...,-0.08,4.262673,Tier 2,None,None,bryan ng,None,"[bryan, ng]",nan,[2000-10-11 00:00:00]


In [3131]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')

null.to_csv('null.csv', encoding='utf-8')

In [3128]:
dictionary_dob_clean_name.get('oliver lim tze rong')

'2000-06-19 00:00:00'

In [ ]:
'''
def length(string):

    B = ''
    year = ''

    try:

        length = len(string)

        if length == 2:

            string = '19' + string

        elif length == 1:

            string = ''

        else:

            pass

        if string is not None or len(string) != 1:

            B = parser.parse(string, dayfirst=True)
                        
    except:

        pass

    return B


final_df['DOB_new'] = final['DOB'].apply(length)
'''

In [2568]:
final_df['DOB_dt'] = pd.to_datetime(athletes['DOB_dt'], errors='coerce')

final_df['year_extract'] = final_df['DOB_dt'].dt.strftime('%Y')

final_df['year_extract'] = pd.to_numeric(final_df['year_extract'])

final_df['age_extract'] = 2025 - final_df['year_extract']


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2062779215.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['DOB_new'] = pd.to_datetime(athletes['DOB_new'], errors='coerce')
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2062779215.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['year_extract'] = final_df['DOB_new'].dt.strftime('%Y')
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2062779215.py:5: SettingWithCopyWarning: 
A value i

In [2569]:
def age(number):  # correct negative age numbers
    
    if number<0:
        
        number+=100
        
    return number


final_df['age_extract']=final_df['age_extract'].apply(age)


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/1795310803.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['age_extract']=final_df['age_extract'].apply(age)


In [2570]:
# If NSG event then choose AGE otherwise choose age_extract

condition1 = final_df['COMPETITION']=='National School Games'

final_df['age_extract'] = final_df['AGE'].where((condition1), final_df['age_extract'].values)
final_df[['age_extract']] = final_df[['age_extract']].apply(pd.to_numeric)

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/1013484073.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['age_extract'] = final_df['AGE'].where((condition1), final_df['age_extract'].values)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/1013484073.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df[['age_extract']] = final_df[['age_extract']].apply(pd.to_numeric)


In [2573]:
final_df

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER
1,11022,"Zhong, Chuhan",12.26,None,None,None,None,Triple Jump,None,None,...,11.46420,11.2860,12.26,12.26,0.6176,0.79580,0.9740,0.38,8.198653,Tier 1
47,12316,Tse Teng Tan,11.59,None,None,None,None,Triple Jump,None,None,...,11.46420,11.2860,11.59,11.59,-0.0524,0.12580,0.3040,-0.29,2.558923,Tier 3
57,6870,"Tang Kai Sheng, Cayman",14.17m,VICTORIA SCHOOL,16,1,U18,Triple Jump,0,None,...,13.15295,12.9485,14.17,14.17,0.8126,1.01705,1.2215,0.54,8.961849,Tier 1
61,11299,"Tan, Tse Teng",11.77m,Hwa Chong Alumni Association,12,2,Open,Triple Jump,0,None,...,11.46420,11.2860,11.77,11.77,0.1276,0.30580,0.4840,-0.11,4.074074,Tier 2
79,9726,Tan Shou Yi Rei (Chen Shouyi),14.99m,Raffles Institution JC,17,1,U20,Triple Jump,0,None,...,13.15295,12.9485,14.99,14.99,1.6326,1.83705,2.0415,1.36,14.977990,Tier 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8462,1883,"Bin Zainal, Muhammad Fazrey",11.32,Singapore Institute of Technol,12,17,Open,Dash,100,,...,11.22975,11.3925,11.32,11.32,-0.2530,-0.09025,0.0725,-0.47,0.668203,Tier 4
8506,1370,"Ashok, Tharun",10.81,Ngee Ann Polytechnic,12,4,Open,Dash,100,,...,11.22975,11.3925,10.81,10.81,0.2570,0.41975,0.5825,0.04,5.368664,Tier 1
8533,889,"Ang, Jamie El-Red",11.09,Oldham Athletics,12,19,Open,Dash,100,None,...,11.22975,11.3925,11.09,11.09,-0.0230,0.13975,0.3025,-0.24,2.788018,Tier 3
8556,3341,Aidan Michael Soh,11.16,INDIVIDUAL,18,None,None,100m,None,None,...,11.22975,11.3925,11.16,11.16,-0.0930,0.06975,0.2325,-0.31,2.142857,Tier 3


In [2574]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


final_df.to_csv('final_df.csv', encoding='utf-8')

In [2578]:
# Filter out nan DOB first and list selection.  We deal with nan DOB next step

selection_with_dob = final_df[(final_df['DOB'])!='None']

selection_with_dob

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER
1,11022,"Zhong, Chuhan",12.26,None,None,None,None,Triple Jump,None,None,...,11.46420,11.2860,12.26,12.26,0.6176,0.79580,0.9740,0.38,8.198653,Tier 1
57,6870,"Tang Kai Sheng, Cayman",14.17m,VICTORIA SCHOOL,16,1,U18,Triple Jump,0,None,...,13.15295,12.9485,14.17,14.17,0.8126,1.01705,1.2215,0.54,8.961849,Tier 1
61,11299,"Tan, Tse Teng",11.77m,Hwa Chong Alumni Association,12,2,Open,Triple Jump,0,None,...,11.46420,11.2860,11.77,11.77,0.1276,0.30580,0.4840,-0.11,4.074074,Tier 2
79,9726,Tan Shou Yi Rei (Chen Shouyi),14.99m,Raffles Institution JC,17,1,U20,Triple Jump,0,None,...,13.15295,12.9485,14.99,14.99,1.6326,1.83705,2.0415,1.36,14.977990,Tier 1
84,9476,"Sueto, Yuito",14.76m,Kansai University of Social We,12,3,Open,Triple Jump,0,None,...,13.15295,12.9485,14.76,14.76,1.4026,1.60705,1.8115,1.13,13.290536,Tier 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8462,1883,"Bin Zainal, Muhammad Fazrey",11.32,Singapore Institute of Technol,12,17,Open,Dash,100,,...,11.22975,11.3925,11.32,11.32,-0.2530,-0.09025,0.0725,-0.47,0.668203,Tier 4
8506,1370,"Ashok, Tharun",10.81,Ngee Ann Polytechnic,12,4,Open,Dash,100,,...,11.22975,11.3925,10.81,10.81,0.2570,0.41975,0.5825,0.04,5.368664,Tier 1
8533,889,"Ang, Jamie El-Red",11.09,Oldham Athletics,12,19,Open,Dash,100,None,...,11.22975,11.3925,11.09,11.09,-0.0230,0.13975,0.3025,-0.24,2.788018,Tier 3
8556,3341,Aidan Michael Soh,11.16,INDIVIDUAL,18,None,None,100m,None,None,...,11.22975,11.3925,11.16,11.16,-0.0930,0.06975,0.2325,-0.31,2.142857,Tier 3


In [ ]:

selection_with_dob[['age_extract']] = selection_with_dob[['age_extract']].apply(pd.to_numeric)

mask = (((selection_with_dob['age_extract'] >= 14) & (selection_with_dob['age_extract'] <= 16))|(selection_with_dob['age_extract'].isna()))
#mask = ((final_df['age_extract'].isnull()))


selection_with_dob = selection_with_dob.loc[mask]


In [2572]:
selection_with_dob

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER
1,11022,"Zhong, Chuhan",12.26,None,None,None,None,Triple Jump,None,None,...,11.46420,11.2860,12.26,12.26,0.6176,0.79580,0.9740,0.38,8.198653,Tier 1
47,12316,Tse Teng Tan,11.59,None,None,None,None,Triple Jump,None,None,...,11.46420,11.2860,11.59,11.59,-0.0524,0.12580,0.3040,-0.29,2.558923,Tier 3
57,6870,"Tang Kai Sheng, Cayman",14.17m,VICTORIA SCHOOL,16,1,U18,Triple Jump,0,None,...,13.15295,12.9485,14.17,14.17,0.8126,1.01705,1.2215,0.54,8.961849,Tier 1
61,11299,"Tan, Tse Teng",11.77m,Hwa Chong Alumni Association,12,2,Open,Triple Jump,0,None,...,11.46420,11.2860,11.77,11.77,0.1276,0.30580,0.4840,-0.11,4.074074,Tier 2
84,9476,"Sueto, Yuito",14.76m,Kansai University of Social We,12,3,Open,Triple Jump,0,None,...,13.15295,12.9485,14.76,14.76,1.4026,1.60705,1.8115,1.13,13.290536,Tier 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8448,10792,Bryan Ng,10.93,None,None,2,None,100m,None,None,...,11.22975,11.3925,10.93,10.93,0.1370,0.29975,0.4625,-0.08,4.262673,Tier 2
8462,1883,"Bin Zainal, Muhammad Fazrey",11.32,Singapore Institute of Technol,12,17,Open,Dash,100,,...,11.22975,11.3925,11.32,11.32,-0.2530,-0.09025,0.0725,-0.47,0.668203,Tier 4
8506,1370,"Ashok, Tharun",10.81,Ngee Ann Polytechnic,12,4,Open,Dash,100,,...,11.22975,11.3925,10.81,10.81,0.2570,0.41975,0.5825,0.04,5.368664,Tier 1
8533,889,"Ang, Jamie El-Red",11.09,Oldham Athletics,12,19,Open,Dash,100,None,...,11.22975,11.3925,11.09,11.09,-0.0230,0.13975,0.3025,-0.24,2.788018,Tier 3
